# Climate Modeling: From Simple Energy Balance to GraphCast

**A Progressive Journey Through Five Climate Models**

This notebook explores climate modeling through five progressively sophisticated approaches, culminating in Google's GraphCast. Each model builds upon the previous one, adding complexity and realism while maintaining scientific rigor.

## Overview

Climate models are mathematical representations of Earth's climate system. They range from simple energy balance equations to complex machine learning systems that can forecast weather patterns. This notebook presents:

1. **Zero-Dimensional Energy Balance Model** - The foundation of climate science
2. **One-Dimensional Radiative-Convective Model** - Adding vertical atmospheric structure  
3. **Two-Dimensional Statistical Dynamical Model** - Including latitude variations
4. **Three-Dimensional General Circulation Model** - Full spatial dynamics
5. **GraphCast-Style ML Model** - Modern AI/ML approach to weather/climate prediction

Each model includes:
- Detailed technical explanation (2 pages) of assumptions and approximations
- Implementation with documented code
- Visualizations of key results
- Analysis of climate change implications

---

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import odeint, solve_ivp
from scipy.optimize import fsolve, minimize
import pandas as pd
from typing import Tuple, List, Callable
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
%matplotlib inline

print("✓ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")

<a id='model1'></a>
## Model 1: Zero-Dimensional Energy Balance Model (EBM)

### Technical Overview (Page 1 of 2)

The Zero-Dimensional Energy Balance Model represents Earth as a single point with no spatial variation. Despite its simplicity, it captures the fundamental physics governing Earth's temperature: the balance between incoming solar radiation and outgoing infrared radiation.

#### Fundamental Equation

The governing equation is:

$$C \frac{dT}{dt} = Q(1-\alpha) - \epsilon\sigma T^4 + F$$

Where:
- $C$ = Climate system heat capacity (J m⁻² K⁻¹) ≈ 10⁸ J m⁻² K⁻¹
- $T$ = Global mean surface temperature (K)
- $Q$ = Incoming solar radiation per unit area = S₀/4 ≈ 342 W m⁻²
- $\alpha$ = Planetary albedo (reflectivity) ≈ 0.30
- $\epsilon$ = Effective emissivity ≈ 0.61 (accounting for greenhouse effect)
- $\sigma$ = Stefan-Boltzmann constant = 5.67 × 10⁻⁸ W m⁻² K⁻⁴
- $F$ = Additional radiative forcing (W m⁻²)

#### Key Physical Assumptions

1. **Spatial Homogeneity**: Earth is treated as a uniform sphere with no variation in latitude, longitude, or altitude. All locations have identical temperature and properties.

2. **Radiative Equilibrium**: The climate is determined entirely by radiative processes. Heat transport by atmosphere and oceans is implicitly included in the effective heat capacity.

3. **Gray Atmosphere**: The atmosphere absorbs and emits radiation uniformly across all wavelengths, simplified into a single emissivity parameter.

4. **Blackbody Radiation**: Earth's surface and atmosphere emit according to the Stefan-Boltzmann law with modification by emissivity.

5. **Steady-State Geometry**: The factor of 4 in $Q = S_0/4$ comes from the ratio of Earth's cross-sectional area (πR²) to total surface area (4πR²).

6. **Linear Heat Capacity**: The relationship between energy storage and temperature change is linear and constant.

### Technical Overview (Page 2 of 2)

#### Mathematical Approximations

**Greenhouse Effect Parameterization**: The most significant approximation is representing the complex greenhouse effect (involving multiple gases with wavelength-dependent absorption) as a single emissivity parameter $\epsilon$. In reality:

- Different greenhouse gases (H₂O, CO₂, CH₄, N₂O) absorb at different wavelengths
- Atmospheric temperature profile affects emission altitude  
- Cloud effects are highly variable
- The model captures this complexity through $\epsilon ≈ 0.61$, calibrated to match observed Earth temperature

**Heat Capacity Lumping**: The ocean mixed layer, land surface, deep ocean, and atmosphere have vastly different heat capacities and response times (hours to millennia). The model uses an effective value representing primarily the ocean mixed layer (~50-100m depth).

**Albedo Simplification**: Planetary albedo varies with:
- Ice cover (0.5-0.9)
- Clouds (0.4-0.9)  
- Vegetation (0.1-0.2)
- Ocean (0.06)

The constant $\alpha = 0.30$ is a global annual mean that changes with climate.

#### Climate Sensitivity

At equilibrium ($dT/dt = 0$), the temperature is:

$$T_{eq} = \left(\frac{Q(1-\alpha) + F}{\epsilon\sigma}\right)^{1/4}$$

The **equilibrium climate sensitivity** (ECS) - temperature change for doubled CO₂ - can be calculated. Doubling CO₂ produces forcing $\Delta F ≈ 3.7-4.0$ W m⁻², yielding:

$$\Delta T_{eq} = T_{eq}(F + \Delta F) - T_{eq}(F)$$

In this simple model, ECS ≈ 1.2°C, which is lower than the IPCC range of 2.5-4°C because the model lacks important positive feedbacks:
- Water vapor feedback (warming → more H₂O → more greenhouse effect)
- Ice-albedo feedback (warming → less ice → less reflection → more warming)
- Cloud feedbacks (complex, both positive and negative)

#### Limitations

1. **No Geography**: Cannot represent land-ocean contrasts, mountain effects, or regional climate
2. **No Seasons**: Annual mean only; cannot capture seasonal cycle or extreme events
3. **No Dynamics**: Atmospheric and oceanic circulation ignored
4. **No Weather**: All synoptic-scale variability averaged out
5. **Underestimates Sensitivity**: Missing key positive feedbacks
6. **No Hydrological Cycle**: Precipitation and evaporation not represented

#### Strengths and Use Cases

Despite limitations, this model:
- ✓ Correctly predicts Earth's mean temperature (~288 K vs observed)
- ✓ Demonstrates fundamental greenhouse effect
- ✓ Shows qualitative response to forcing changes
- ✓ Provides physical intuition for energy balance
- ✓ Fast computation for parameter sensitivity studies
- ✓ Good first-order estimate of climate sensitivity

**Applications**: Education, rapid scenario testing, understanding basic climate physics, validating more complex models.

In [ ]:
class ZeroDimensionalEBM:
    """
    Zero-Dimensional Energy Balance Model
    
    Solves: C*dT/dt = Q*(1-α) - ε*σ*T^4 + F
    
    where Earth is treated as a single point with uniform temperature.
    """
    
    def __init__(self, C=1e8, alpha=0.30, epsilon=0.61):
        """
        Initialize model with physical constants
        
        Parameters:
        -----------
        C : float
            Heat capacity (J m^-2 K^-1), default 1e8 (ocean mixed layer ~100m)
        alpha : float  
            Planetary albedo (dimensionless), default 0.30
        epsilon : float
            Effective emissivity (dimensionless), default 0.61
        """
        # Physical constants (SI units)
        self.sigma = 5.67e-8  # Stefan-Boltzmann constant (W m^-2 K^-4)
        self.S0 = 1361.0      # Solar constant at Earth (W m^-2)
        self.Q = self.S0 / 4  # Average incoming solar (geometry factor)
        
        # Model parameters
        self.C = C
        self.alpha = alpha
        self.epsilon = epsilon
        
    def absorbed_solar(self):
        """Calculate absorbed solar radiation (W m^-2)"""
        return self.Q * (1 - self.alpha)
    
    def emitted_ir(self, T):
        """Calculate emitted infrared radiation (W m^-2)"""
        return self.epsilon * self.sigma * T**4
    
    def net_radiation(self, T, forcing=0):
        """Calculate net radiative balance (W m^-2)"""
        return self.absorbed_solar() + forcing - self.emitted_ir(T)
    
    def dT_dt(self, T, t, forcing=0):
        """
        Temperature tendency equation
        
        Returns dT/dt in K/year
        """
        dE_dt = self.net_radiation(T, forcing)  # W m^-2
        seconds_per_year = 365.25 * 24 * 3600
        return (dE_dt / self.C) * seconds_per_year
    
    def equilibrium_temperature(self, forcing=0):
        """
        Calculate equilibrium temperature analytically
        
        Parameters:
        -----------
        forcing : float
            Additional radiative forcing (W m^-2)
            
        Returns:
        --------
        T_eq : float
            Equilibrium temperature (K)
        """
        numerator = self.absorbed_solar() + forcing
        T_eq = (numerator / (self.epsilon * self.sigma))**0.25
        return T_eq
    
    def run_simulation(self, T0, years, forcing=0, dt=0.1):
        """
        Time-dependent simulation
        
        Parameters:
        -----------
        T0 : float
            Initial temperature (K)
        years : float
            Simulation duration (years)
        forcing : float or callable
            Constant forcing (W m^-2) or function f(t) returning forcing
        dt : float
            Time step (years)
            
        Returns:
        --------
        t : ndarray
            Time points (years)
        T : ndarray
            Temperature evolution (K)
        """
        t = np.arange(0, years, dt)
        T = np.zeros_like(t)
        T[0] = T0
        
        # Check if forcing is callable
        if callable(forcing):
            forcing_func = forcing
        else:
            forcing_func = lambda t: forcing
        
        # Forward Euler integration (simple and stable for this problem)
        for i in range(1, len(t)):
            F_current = forcing_func(t[i-1])
            T[i] = T[i-1] + self.dT_dt(T[i-1], t[i-1], F_current) * dt
            
        return t, T
    
    def climate_sensitivity(self, forcing_2xCO2=3.7):
        """
        Calculate equilibrium climate sensitivity
        
        Parameters:
        -----------
        forcing_2xCO2 : float
            Radiative forcing from doubling CO2 (W m^-2), default 3.7
            
        Returns:
        --------
        ECS : float
            Equilibrium climate sensitivity (K)
        """
        T_current = self.equilibrium_temperature(0)
        T_2xCO2 = self.equilibrium_temperature(forcing_2xCO2)
        return T_2xCO2 - T_current

# Initialize the model
print("="*60)
print("ZERO-DIMENSIONAL ENERGY BALANCE MODEL")
print("="*60 + "\n")

model1 = ZeroDimensionalEBM()

print(f"Physical Constants:")
print(f"  Solar constant (S₀): {model1.S0:.1f} W/m²")
print(f"  Mean solar input (Q): {model1.Q:.1f} W/m²")
print(f"  Stefan-Boltzmann (σ): {model1.sigma:.2e} W/m²/K⁴\n")

print(f"Model Parameters:")
print(f"  Heat capacity (C): {model1.C:.2e} J/m²/K")
print(f"  Albedo (α): {model1.alpha:.2f}")
print(f"  Emissivity (ε): {model1.epsilon:.2f}\n")

# Calculate current climate equilibrium
T_eq = model1.equilibrium_temperature()
print(f"Current Climate:")
print(f"  Equilibrium temperature: {T_eq:.2f} K ({T_eq-273.15:.2f}°C)")
print(f"  Absorbed solar: {model1.absorbed_solar():.1f} W/m²")
print(f"  Emitted IR: {model1.emitted_ir(T_eq):.1f} W/m²\n")

# Calculate climate sensitivity
ECS = model1.climate_sensitivity()
print(f"Climate Sensitivity:")
print(f"  2×CO₂ forcing: 3.7 W/m²")
print(f"  Equilibrium climate sensitivity: {ECS:.2f} K")
print(f"  New equilibrium: {T_eq+ECS:.2f} K ({T_eq+ECS-273.15:.2f}°C)")
print("\n" + "="*60)

In [ ]:
# Create comprehensive visualizations for Model 1

fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# === Panel 1: Energy Balance Diagram ===
ax1 = fig.add_subplot(gs[0, :2])
T_range = np.linspace(240, 320, 200)
Q_in = model1.absorbed_solar()
Q_out = model1.emitted_ir(T_range)

ax1.plot(T_range-273.15, Q_in*np.ones_like(T_range), 'r-', linewidth=3, 
         label='Absorbed Solar Radiation', alpha=0.8)
ax1.plot(T_range-273.15, Q_out, 'b-', linewidth=3, 
         label='Emitted Infrared Radiation', alpha=0.8)
ax1.axvline(T_eq-273.15, color='green', linestyle='--', linewidth=2, 
            label=f'Equilibrium ({T_eq-273.15:.1f}°C)', alpha=0.7)
ax1.fill_between(T_range-273.15, Q_in*np.ones_like(T_range), Q_out, 
                  where=(Q_in >= Q_out), alpha=0.2, color='red', label='Warming')
ax1.fill_between(T_range-273.15, Q_in*np.ones_like(T_range), Q_out,
                  where=(Q_in < Q_out), alpha=0.2, color='blue', label='Cooling')

ax1.set_xlabel('Temperature (°C)', fontsize=13, fontweight='bold')
ax1.set_ylabel('Radiation (W/m²)', fontsize=13, fontweight='bold')
ax1.set_title('Model 1: Energy Balance Diagram', fontsize=15, fontweight='bold')
ax1.legend(fontsize=10, loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-30, 45)
ax1.set_ylim(150, 450)

# === Panel 2: Parameter Sensitivity ===
ax2 = fig.add_subplot(gs[0, 2])
alphas = np.linspace(0.1, 0.5, 50)
T_alpha = [(model1.Q * (1-a) / (model1.epsilon * model1.sigma))**0.25 - 273.15 
           for a in alphas]

ax2.plot(alphas, T_alpha, 'purple', linewidth=3)
ax2.axvline(model1.alpha, color='red', linestyle='--', alpha=0.7, 
            label=f'Current α={model1.alpha}')
ax2.axhline(T_eq-273.15, color='green', linestyle='--', alpha=0.7)
ax2.set_xlabel('Albedo α', fontsize=11, fontweight='bold')
ax2.set_ylabel('Equilibrium T (°C)', fontsize=11, fontweight='bold')
ax2.set_title('Albedo Sensitivity', fontsize=13, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# === Panel 3: Temperature Evolution (Cold Start) ===
ax3 = fig.add_subplot(gs[1, 0])
t1, T1 = model1.run_simulation(T0=250, years=100, dt=0.1)
ax3.plot(t1, T1-273.15, 'b-', linewidth=2.5, label='Cold start (250 K)')
ax3.axhline(T_eq-273.15, color='green', linestyle='--', linewidth=2, 
            alpha=0.7, label='Equilibrium')
ax3.set_xlabel('Time (years)', fontsize=11, fontweight='bold')
ax3.set_ylabel('Temperature (°C)', fontsize=11, fontweight='bold')
ax3.set_title('Cold Start Response', fontsize=13, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0, 100)

# === Panel 4: Temperature Evolution (Warm Start) ===
ax4 = fig.add_subplot(gs[1, 1])
t2, T2 = model1.run_simulation(T0=310, years=100, dt=0.1)
ax4.plot(t2, T2-273.15, 'r-', linewidth=2.5, label='Warm start (310 K)')
ax4.axhline(T_eq-273.15, color='green', linestyle='--', linewidth=2,
            alpha=0.7, label='Equilibrium')
ax4.set_xlabel('Time (years)', fontsize=11, fontweight='bold')
ax4.set_ylabel('Temperature (°C)', fontsize=11, fontweight='bold')
ax4.set_title('Warm Start Response', fontsize=13, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)
ax4.set_xlim(0, 100)

# === Panel 5: Climate Forcing Response ===
ax5 = fig.add_subplot(gs[1, 2])
forcings = np.linspace(-10, 10, 100)
T_forced = [model1.equilibrium_temperature(f) - 273.15 for f in forcings]

ax5.plot(forcings, T_forced, 'darkgreen', linewidth=3)
ax5.axvline(0, color='gray', linestyle='--', alpha=0.5)
ax5.axvline(3.7, color='red', linestyle=':', linewidth=2, 
            alpha=0.7, label='2×CO₂ (~3.7 W/m²)')
ax5.axhline(T_eq-273.15, color='gray', linestyle='--', alpha=0.5)
ax5.set_xlabel('Forcing (W/m²)', fontsize=11, fontweight='bold')
ax5.set_ylabel('Equilibrium T (°C)', fontsize=11, fontweight='bold')
ax5.set_title('Forcing Sensitivity', fontsize=13, fontweight='bold')
ax5.legend(fontsize=9)
ax5.grid(True, alpha=0.3)

# === Panel 6: CO2 Increase Scenario ===
ax6 = fig.add_subplot(gs[2, :])

# Define gradual CO2 increase
def co2_forcing(t):
    """Forcing that ramps up linearly over 100 years to 2xCO2"""
    return min(3.7 * t / 100, 3.7)

t_co2, T_co2 = model1.run_simulation(T0=288, years=200, forcing=co2_forcing, dt=0.1)
forcing_trajectory = np.array([co2_forcing(ti) for ti in t_co2])

# Plot on dual axes
color1 = 'tab:blue'
ax6.set_xlabel('Time (years)', fontsize=13, fontweight='bold')
ax6.set_ylabel('Temperature (°C)', fontsize=13, fontweight='bold', color=color1)
line1 = ax6.plot(t_co2, T_co2-273.15, color=color1, linewidth=3, 
                 label='Global Temperature')
ax6.tick_params(axis='y', labelcolor=color1)
ax6.axhline(T_eq-273.15, color='gray', linestyle='--', alpha=0.4, label='Pre-forcing equilibrium')

ax6_twin = ax6.twinx()
color2 = 'tab:red'
ax6_twin.set_ylabel('CO₂ Forcing (W/m²)', fontsize=13, fontweight='bold', color=color2)
line2 = ax6_twin.plot(t_co2, forcing_trajectory, color=color2, linewidth=2.5, 
                       linestyle='--', alpha=0.7, label='Radiative Forcing')
ax6_twin.tick_params(axis='y', labelcolor=color2)

# Combine legends
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax6.legend(lines, labels, fontsize=11, loc='upper left', framealpha=0.9)
ax6.grid(True, alpha=0.3)
ax6.set_title('Response to Gradual CO₂ Increase (Doubling over 100 years)', 
              fontsize=15, fontweight='bold')
ax6.set_xlim(0, 200)

plt.suptitle('Zero-Dimensional Energy Balance Model: Complete Analysis', 
             fontsize=17, fontweight='bold', y=0.995)

plt.savefig('model1_complete.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("KEY INSIGHTS FROM MODEL 1")
print("="*60)
print("\n1. Energy Balance: Earth maintains equilibrium when absorbed")
print("   solar radiation equals emitted infrared radiation")
print(f"\n2. Current equilibrium: {T_eq-273.15:.1f}°C is very close to")
print("   observed global mean temperature (~15°C)")
print(f"\n3. Climate Sensitivity: Doubling CO₂ (~3.7 W/m²) causes")
print(f"   ~{ECS:.1f}°C warming in this simple model")
print("\n4. Thermal Inertia: Temperature changes lag forcing due to")
print("   ocean heat capacity (time constant ~decades)")
print("\n5. Limitations: This model underestimates sensitivity because")
print("   it lacks key feedbacks (water vapor, ice-albedo, clouds)")
print("\n" + "="*60)

<a id='model2'></a>
## Model 2: One-Dimensional Radiative-Convective Model

### Technical Overview (Page 1 of 2)

The One-Dimensional Radiative-Convective Model extends the zero-dimensional model by adding vertical atmospheric structure. This captures the critical feature that Earth's atmosphere is not uniform - temperature, pressure, and composition vary dramatically with altitude.

#### Governing Equations

The model solves radiative transfer and convective adjustment in a vertical column:

**Radiative Transfer:**
$$\frac{d F_{\uparrow}}{dz} = -\kappa(z)\rho(z)[B(T(z)) - F_{\uparrow}]$$
$$\frac{d F_{\downarrow}}{dz} = \kappa(z)\rho(z)[B(T(z)) - F_{\downarrow}]$$

**Energy Balance:**
$$\rho(z) c_p \frac{\partial T}{\partial t} = -\frac{\partial F_{net}}{\partial z} + Q_{conv}$$

**Convective Adjustment:**
$$\text{If } \frac{dT}{dz} < -\Gamma_{crit}, \text{ adjust to } \frac{dT}{dz} = -\Gamma_{crit}$$

Where:
- $F_{\uparrow}, F_{\downarrow}$ = Upward and downward radiative fluxes (W m⁻²)
- $z$ = Altitude (m)
- $\kappa(z)$ = Absorption coefficient (m² kg⁻¹), varies with wavelength and species
- $\rho(z)$ = Air density (kg m⁻³)
- $B(T)$ = Planck function ≈ $\sigma T^4$ (gray atmosphere approximation)
- $T(z)$ = Temperature profile (K)
- $c_p$ = Specific heat at constant pressure = 1004 J kg⁻¹ K⁻¹
- $Q_{conv}$ = Convective heat flux (W m⁻³)
- $\Gamma_{crit}$ = Critical lapse rate ≈ 6.5 K km⁻¹

#### Key Physical Assumptions

1. **One-Dimensional**: Horizontal homogeneity - no variation in x or y directions. Represents a global or zonal mean.

2. **Hydrostatic Balance**: Pressure decreases exponentially with altitude according to $P(z) = P_0 e^{-z/H}$ where $H ≈ 8$ km is the scale height.

3. **Gray Atmosphere**: Absorption and emission are wavelength-independent, characterized by a single optical depth $\tau$.

4. **Two-Stream Approximation**: Radiation is either purely upward or purely downward, neglecting sideways scattering.

5. **Schwarzschild Equation**: Each atmospheric layer emits as a blackbody and absorbs radiation passing through it.

6. **Convective Adjustment**: When radiative equilibrium produces a superadiabatic lapse rate (unstable), convection instantly adjusts the profile to the critical lapse rate.

#### Vertical Structure

The atmosphere is divided into layers (typically 20-50):
- **Troposphere** (0-12 km): Temperature decreases with height, governed by moist convection
- **Stratosphere** (12-50 km): Temperature increases with height due to ozone absorption (simplified or omitted in basic versions)
- **Surface**: Coupled to lowest atmospheric layer via radiation and turbulent fluxes

### Technical Overview (Page 2 of 2)

#### Mathematical Approximations

**Gray Atmosphere Approximation**: Real greenhouse gases have complex, wavelength-dependent absorption:
- H₂O absorbs strongly at 6.3 µm (vibration-rotation) and >15 µm (pure rotation)
- CO₂ absorbs at 15 µm and 4.3 µm
- O₃ absorbs in UV and at 9.6 µm
- Clouds absorb and scatter across broad spectrum

The gray approximation uses effective optical depth:
$$\tau_{eff} = \int_0^{\infty} \kappa_\lambda(z) \rho(z) dz$$

calibrated to match observed radiative fluxes. Typical values: $\tau_{eff} ≈ 1-2$ for clear sky.

**Two-Stream Radiative Transfer**: The full radiative transfer equation is an integro-differential equation accounting for scattering in all directions. The two-stream approximation assumes:
- Upward flux: $F_{\uparrow}(z) = \pi I_{\uparrow}$ (hemispheric integral)
- Downward flux: $F_{\downarrow}(z) = \pi I_{\downarrow}$

This is accurate to within ~10-20% for thermal radiation but less accurate for solar radiation with scattering.

**Convective Parameterization**: Real atmospheric convection involves:
- Cloud formation and latent heat release
- Entrainment and detrainment
- Mesoscale organization
- Turbulent eddies

The model uses instantaneous adjustment to a prescribed lapse rate:
$$\Gamma = \Gamma_d \frac{1 + L_v q_s / (R_d T)}{1 + L_v^2 q_s / (c_p R_v T^2)} ≈ 6.5 \text{ K/km}$$

where $\Gamma_d = g/c_p ≈ 9.8$ K/km is the dry adiabatic lapse rate, modified by moisture.

**Solar Absorption**: Simplified to:
- Surface absorbs most solar radiation
- Stratospheric ozone absorption neglected or parameterized
- Cloud effects on solar radiation simplified

#### Radiative-Convective Equilibrium

The model seeks equilibrium where:
1. Surface energy budget balances: solar absorption = IR emission + sensible heat
2. Each atmospheric layer has zero net radiative heating or is convectively neutral
3. Top-of-atmosphere energy budget closes

The equilibrium is found iteratively:
1. Calculate radiative fluxes for given T(z)
2. Compute radiative heating rates
3. Update T(z) toward radiative equilibrium
4. Apply convective adjustment where unstable
5. Repeat until convergence

#### Improvements Over Model 1

✓ **Vertical temperature structure**: Captures troposphere-stratosphere distinction
✓ **Atmospheric greenhouse effect**: Explicitly represents radiation absorption/emission by gases
✓ **Lapse rate feedback**: Changes in vertical temperature profile affect sensitivity  
✓ **Surface-atmosphere coupling**: Distinguishes surface from atmospheric temperatures
✓ **Altitude-dependent forcing**: CO₂ forcing affects different layers differently

#### Remaining Limitations

✗ **No horizontal structure**: Cannot represent equator-pole temperature gradient
✗ **No dynamics**: Winds and pressure systems not included
✗ **No clouds**: Major uncertainty in real climate
✗ **No seasons**: Time-mean only
✗ **Simplified convection**: Real convection is complex and localized

#### Climate Sensitivity

In radiative-convective models, ECS ≈ 1.5-2.5°C, closer to observations than Model 1 because:
- Water vapor feedback included: warmer atmosphere holds more H₂O
- Lapse rate feedback: tropospheric warming pattern affects surface response
- Still missing ice-albedo, cloud feedbacks

In [ ]:
class OneDimensionalRCM:
    """
    One-Dimensional Radiative-Convective Model
    
    Solves radiative transfer and convective adjustment in a vertical column.
    Represents vertical atmospheric structure from surface to top of atmosphere.
    """
    
    def __init__(self, n_levels=30, p_surface=1013.25, p_top=10.0):
        """
        Initialize 1D radiative-convective model
        
        Parameters:
        -----------
        n_levels : int
            Number of vertical levels
        p_surface : float
            Surface pressure (hPa)
        p_top : float
            Top of atmosphere pressure (hPa)
        """
        # Physical constants
        self.g = 9.81           # Gravity (m/s²)
        self.cp = 1004.0        # Specific heat at const pressure (J/kg/K)
        self.R = 287.0          # Gas constant for dry air (J/kg/K)
        self.sigma = 5.67e-8    # Stefan-Boltzmann constant
        self.S0 = 1361.0        # Solar constant (W/m²)
        
        # Model parameters
        self.albedo = 0.30      # Planetary albedo
        self.tau_lw = 1.5       # Longwave optical depth
        self.solar_abs_atm = 0.2  # Fraction of solar absorbed by atmosphere
        self.critical_lapse = 6.5e-3  # Critical lapse rate (K/m)
        
        # Vertical grid
        self.n_levels = n_levels
        self.p_surface = p_surface  # hPa
        self.p_top = p_top          # hPa
        
        # Create pressure levels (equally spaced in log-pressure)
        self.p = np.logspace(np.log10(p_top), np.log10(p_surface), n_levels)  # hPa
        self.p_pa = self.p * 100  # Convert to Pa
        
        # Calculate layer properties
        self.dp = np.diff(self.p_pa)  # Pressure thickness of layers
        self.z = self._pressure_to_height(self.p_pa)  # Approximate heights
        
    def _pressure_to_height(self, p):
        """Convert pressure to approximate height using hydrostatic equation"""
        H = self.R * 250 / self.g  # Scale height (~7.5 km for T=250K)
        return -H * np.log(p / (self.p_surface * 100))
    
    def _height_to_temperature(self, z, T_surface):
        """Standard atmosphere approximation"""
        # Troposphere: linear decrease
        T = T_surface - self.critical_lapse * z
        # Don't let temperature go below 180 K (stratosphere)
        return np.maximum(T, 180)
    
    def planck_emission(self, T):
        """Blackbody emission (W/m²)"""
        return self.sigma * T**4
    
    def optical_depth_profile(self):
        """
        Calculate optical depth at each level
        Increases with pressure (more gas below)
        """
        # Optical depth increases toward surface
        tau = self.tau_lw * (self.p / self.p_surface)
        return tau
    
    def compute_radiative_fluxes(self, T):
        """
        Compute upward and downward longwave fluxes using two-stream approximation
        
        Parameters:
        -----------
        T : array
            Temperature at each level (K)
            
        Returns:
        --------
        F_up : array
            Upward flux at each level (W/m²)
        F_down : array
            Downward flux at each level (W/m²)
        """
        n = len(T)
        F_up = np.zeros(n+1)    # Fluxes at layer interfaces
        F_down = np.zeros(n+1)
        
        tau = self.optical_depth_profile()
        
        # Surface emission (bottom boundary)
        F_up[0] = self.planck_emission(T[0])
        
        # Upward flux: integrate from surface to top
        for i in range(n):
            B_i = self.planck_emission(T[i])
            if i < n-1:
                dtau = tau[i] - tau[i+1]
            else:
                dtau = tau[i]
            
            # Two-stream approximation
            transmittance = np.exp(-dtau)
            F_up[i+1] = F_up[i] * transmittance + B_i * (1 - transmittance)
        
        # Downward flux: integrate from top to surface  
        F_down[-1] = 0  # No downward flux at TOA
        
        for i in range(n-1, -1, -1):
            B_i = self.planck_emission(T[i])
            if i > 0:
                dtau = tau[i] - tau[i-1]
            else:
                dtau = tau[i]
            
            transmittance = np.exp(-dtau)
            F_down[i] = F_down[i+1] * transmittance + B_i * (1 - transmittance)
        
        return F_up, F_down
    
    def solar_heating(self, T):
        """
        Calculate solar heating rate in each layer
        
        Returns:
        --------
        Q_solar : array
            Heating rate (K/day) for each level
        """
        Q_in = (self.S0 / 4) * (1 - self.albedo)  # Absorbed solar
        
        # Simple distribution: most at surface, some in atmosphere
        Q_solar = np.zeros(self.n_levels)
        
        # Atmospheric absorption (decreases exponentially upward)
        for i in range(self.n_levels):
            altitude_factor = np.exp(-(self.n_levels - i) / 10)
            Q_solar[i] = Q_in * self.solar_abs_atm * altitude_factor
        
        # Surface gets the rest
        Q_solar[0] += Q_in * (1 - self.solar_abs_atm)
        
        # Convert to heating rate (K/day)
        mass_per_area = self.p_pa / self.g  # kg/m²
        seconds_per_day = 86400
        
        for i in range(self.n_levels):
            if i == 0:
                dm = mass_per_area[0]
            else:
                dm = abs(mass_per_area[i] - mass_per_area[i-1])
            
            if dm > 0:
                Q_solar[i] = (Q_solar[i] / (dm * self.cp)) * seconds_per_day
        
        return Q_solar
    
    def longwave_heating(self, F_up, F_down):
        """
        Calculate longwave radiative heating rate
        
        Returns:
        --------
        Q_lw : array  
            Cooling rate (K/day) for each level
        """
        Q_lw = np.zeros(self.n_levels)
        
        # Heating = convergence of net flux
        F_net = F_up - F_down
        
        mass_per_area = self.p_pa / self.g
        seconds_per_day = 86400
        
        for i in range(self.n_levels):
            # Flux convergence
            if i == 0:
                dF = F_net[1] - F_net[0]
                dm = mass_per_area[0]
            elif i == self.n_levels - 1:
                dF = F_net[i+1] - F_net[i]
                dm = abs(mass_per_area[i] - mass_per_area[i-1])
            else:
                dF = F_net[i+1] - F_net[i]  
                dm = abs(mass_per_area[i] - mass_per_area[i-1])
            
            if dm > 0:
                Q_lw[i] = -(dF / (dm * self.cp)) * seconds_per_day
        
        return Q_lw
    
    def apply_convective_adjustment(self, T):
        """
        Adjust temperature profile to critical lapse rate where unstable
        
        Parameters:
        -----------
        T : array
            Temperature profile (K)
            
        Returns:
        --------
        T_adjusted : array
            Adjusted temperature profile (K)
        """
        T_adj = T.copy()
        
        # Check lapse rate from surface upward
        for i in range(len(T) - 1):
            if self.z[i+1] > self.z[i]:  # Make sure height increases
                dz = self.z[i+1] - self.z[i]
                actual_lapse = -(T_adj[i+1] - T_adj[i]) / dz
                
                # If super-adiabatic (too steep), adjust
                if actual_lapse > self.critical_lapse:
                    # Set to critical lapse rate
                    T_adj[i+1] = T_adj[i] - self.critical_lapse * dz
        
        return T_adj
    
    def run_to_equilibrium(self, T_initial=None, max_iterations=1000, 
                          tolerance=0.01, forcing=0):
        """
        Iterate to radiative-convective equilibrium
        
        Parameters:
        -----------
        T_initial : array, optional
            Initial temperature profile (K). If None, uses standard atmosphere.
        max_iterations : int
            Maximum iterations
        tolerance : float  
            Convergence criterion (K)
        forcing : float
            Additional radiative forcing (W/m²) at surface
            
        Returns:
        --------
        T : array
            Equilibrium temperature profile (K)
        converged : bool
            Whether solution converged
        """
        # Initialize temperature profile
        if T_initial is None:
            T_surface_guess = 288  # K
            T = self._height_to_temperature(self.z, T_surface_guess)
        else:
            T = T_initial.copy()
        
        # Relaxation parameter for stability
        alpha = 0.1
        
        for iteration in range(max_iterations):
            T_old = T.copy()
            
            # Compute radiative fluxes
            F_up, F_down = self.compute_radiative_fluxes(T)
            
            # Add forcing to surface
            F_up[0] += forcing
            
            # Compute heating rates
            Q_solar = self.solar_heating(T)
            Q_lw = self.longwave_heating(F_up, F_down)
            Q_total = Q_solar + Q_lw
            
            # Update temperature
            T = T + alpha * Q_total
            
            # Apply convective adjustment
            T = self.apply_convective_adjustment(T)
            
            # Check convergence
            max_change = np.max(np.abs(T - T_old))
            if max_change < tolerance:
                return T, True, iteration
        
        return T, False, max_iterations
    
    def climate_sensitivity(self, forcing_2xCO2=4.0):
        """
        Calculate equilibrium climate sensitivity
        
        Returns:
        --------
        T_control : array
            Control climate temperature profile
        T_2xCO2 : array
            2×CO₂ temperature profile  
        ECS : float
            Equilibrium climate sensitivity (surface temperature change)
        """
        # Control climate
        T_control, _, _ = self.run_to_equilibrium(forcing=0)
        
        # 2×CO₂ climate
        T_2xCO2, _, _ = self.run_to_equilibrium(forcing=forcing_2xCO2)
        
        ECS = T_2xCO2[0] - T_control[0]
        
        return T_control, T_2xCO2, ECS

# Initialize and run Model 2
print("="*70)
print("ONE-DIMENSIONAL RADIATIVE-CONVECTIVE MODEL")
print("="*70 + "\n")

model2 = OneDimensionalRCM(n_levels=30)

print(f"Model Configuration:")
print(f"  Vertical levels: {model2.n_levels}")
print(f"  Pressure range: {model2.p_top:.1f} - {model2.p_surface:.1f} hPa")
print(f"  Height range: {model2.z[-1]/1000:.1f} - {model2.z[0]/1000:.1f} km")
print(f"  Critical lapse rate: {model2.critical_lapse*1000:.1f} K/km\n")

print("Computing radiative-convective equilibrium...")
T_eq, converged, iterations = model2.run_to_equilibrium()

print(f"  Converged: {converged}")
print(f"  Iterations: {iterations}")
print(f"  Surface temperature: {T_eq[0]:.2f} K ({T_eq[0]-273.15:.2f}°C)")
print(f"  Upper atmosphere: {T_eq[-1]:.2f} K ({T_eq[-1]-273.15:.2f}°C)\n")

print("Computing climate sensitivity...")
T_control, T_2xCO2, ECS = model2.climate_sensitivity()

print(f"  Control surface temp: {T_control[0]:.2f} K ({T_control[0]-273.15:.2f}°C)")
print(f"  2×CO₂ surface temp: {T_2xCO2[0]:.2f} K ({T_2xCO2[0]-273.15:.2f}°C)")
print(f"  Climate sensitivity: {ECS:.2f} K")
print("\n" + "="*70)

In [ ]:
# Visualize Model 2: Radiative-Convective Modelfig = plt.figure(figsize=(16, 14))gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)# === Panel 1: Temperature Profiles ===ax1 = fig.add_subplot(gs[0, 0])ax1.plot(T_control - 273.15, model2.z/1000, 'b-', linewidth=3,          label='Control Climate', marker='o', markersize=4, alpha=0.7)ax1.plot(T_2xCO2 - 273.15, model2.z/1000, 'r-', linewidth=3,         label='2×CO₂ Climate', marker='s', markersize=4, alpha=0.7)# Add standard atmosphere for referenceT_standard = np.array([288 - 6.5*z/1000 for z in model2.z])T_standard = np.maximum(T_standard, 180)ax1.plot(T_standard - 273.15, model2.z/1000, 'k--', linewidth=1.5,         label='Standard Atmosphere', alpha=0.5)ax1.set_xlabel('Temperature (°C)', fontsize=12, fontweight='bold')ax1.set_ylabel('Height (km)', fontsize=12, fontweight='bold')ax1.set_title('Vertical Temperature Profiles', fontsize=14, fontweight='bold')ax1.legend(fontsize=10, loc='upper right')ax1.grid(True, alpha=0.3)ax1.set_ylim(0, 20)ax1.set_xlim(-80, 30)# === Panel 2: Temperature Change ===ax2 = fig.add_subplot(gs[0, 1])delta_T = T_2xCO2 - T_controlax2.plot(delta_T, model2.z/1000, 'purple', linewidth=3, marker='o', markersize=5)ax2.axvline(0, color='gray', linestyle='--', alpha=0.5)ax2.axhline(model2.z[np.argmax(delta_T)]/1000, color='red', linestyle=':',             alpha=0.5, label=f'Max warming: {np.max(delta_T):.2f} K')ax2.set_xlabel('Temperature Change (K)', fontsize=12, fontweight='bold')ax2.set_ylabel('Height (km)', fontsize=12, fontweight='bold')ax2.set_title('Warming Profile (2×CO₂ - Control)', fontsize=14, fontweight='bold')ax2.legend(fontsize=10)ax2.grid(True, alpha=0.3)ax2.set_ylim(0, 20)# === Panel 3: Lapse Rate ===ax3 = fig.add_subplot(gs[1, 0])# Calculate lapse ratesdz = np.diff(model2.z)dT_control = np.diff(T_control)lapse_control = -dT_control / dz * 1000  # K/kmdT_2xco2 = np.diff(T_2xCO2)lapse_2xco2 = -dT_2xco2 / dz * 1000z_mid = (model2.z[:-1] + model2.z[1:]) / 2ax3.plot(lapse_control, z_mid/1000, 'b-', linewidth=2.5,          label='Control', marker='o', markersize=3, alpha=0.7)ax3.plot(lapse_2xco2, z_mid/1000, 'r-', linewidth=2.5,         label='2×CO₂', marker='s', markersize=3, alpha=0.7)ax3.axvline(model2.critical_lapse*1000, color='green', linestyle='--',             linewidth=2, alpha=0.7, label=f'Critical: {model2.critical_lapse*1000:.1f} K/km')ax3.set_xlabel('Lapse Rate (K/km)', fontsize=12, fontweight='bold')ax3.set_ylabel('Height (km)', fontsize=12, fontweight='bold')ax3.set_title('Atmospheric Lapse Rate', fontsize=14, fontweight='bold')ax3.legend(fontsize=10)ax3.grid(True, alpha=0.3)ax3.set_ylim(0, 15)ax3.set_xlim(-10, 12)# === Panel 4: Radiative Fluxes ===ax4 = fig.add_subplot(gs[1, 1])F_up, F_down = model2.compute_radiative_fluxes(T_control)z_flux = np.concatenate([[0], model2.z])ax4.plot(F_up, z_flux/1000, 'r-', linewidth=3, label='Upward LW', marker='^',          markersize=4, alpha=0.7)ax4.plot(F_down, z_flux/1000, 'b-', linewidth=3, label='Downward LW', marker='v',         markersize=4, alpha=0.7)ax4.plot(F_up - F_down, z_flux/1000, 'purple', linewidth=2, linestyle='--',         label='Net Flux', alpha=0.7)ax4.set_xlabel('Radiative Flux (W/m²)', fontsize=12, fontweight='bold')ax4.set_ylabel('Height (km)', fontsize=12, fontweight='bold')ax4.set_title('Longwave Radiative Fluxes', fontsize=14, fontweight='bold')ax4.legend(fontsize=10, loc='best')ax4.grid(True, alpha=0.3)ax4.set_ylim(0, 20)# === Panel 5: Heating Rates ===ax5 = fig.add_subplot(gs[2, 0])Q_solar = model2.solar_heating(T_control)F_up, F_down = model2.compute_radiative_fluxes(T_control)Q_lw = model2.longwave_heating(F_up, F_down)ax5.plot(Q_solar, model2.z/1000, 'orange', linewidth=2.5,          label='Solar Heating', marker='o', markersize=4)ax5.plot(Q_lw, model2.z/1000, 'blue', linewidth=2.5,         label='LW Cooling', marker='s', markersize=4)ax5.plot(Q_solar + Q_lw, model2.z/1000, 'green', linewidth=2.5, linestyle='--',         label='Net Heating', marker='d', markersize=4)ax5.axvline(0, color='gray', linestyle='--', alpha=0.5)ax5.set_xlabel('Heating Rate (K/day)', fontsize=12, fontweight='bold')ax5.set_ylabel('Height (km)', fontsize=12, fontweight='bold')ax5.set_title('Radiative Heating/Cooling Rates', fontsize=14, fontweight='bold')ax5.legend(fontsize=10)ax5.grid(True, alpha=0.3)ax5.set_ylim(0, 15)# === Panel 6: Energy Balance ===ax6 = fig.add_subplot(gs[2, 1])# TOA balanceF_up_toa = F_up[-1]F_solar_in = (model2.S0/4) * (1 - model2.albedo)F_net_toa = F_solar_in - F_up_toa# Surface balanceF_up_sfc = F_up[0]F_down_sfc = F_down[0]F_solar_sfc = F_solar_in * (1 - model2.solar_abs_atm)F_net_sfc = F_solar_sfc + F_down_sfc - F_up_sfccomponents = ['Solar\nIn', 'OLR', 'Net\nTOA', 'Solar\nSfc', 'Down\nLW', 'Up\nLW', 'Net\nSfc']values = [F_solar_in, F_up_toa, F_net_toa, F_solar_sfc, F_down_sfc, -F_up_sfc, F_net_sfc]colors = ['orange', 'blue', 'green', 'orange', 'red', 'blue', 'green']bars = ax6.bar(components, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)# Add value labels on barsfor bar, val in zip(bars, values):    height = bar.get_height()    ax6.text(bar.get_x() + bar.get_width()/2., height,             f'{val:.1f}', ha='center', va='bottom' if height > 0 else 'top',             fontsize=9, fontweight='bold')ax6.axhline(0, color='black', linewidth=1)ax6.set_ylabel('Energy Flux (W/m²)', fontsize=12, fontweight='bold')ax6.set_title('Top-of-Atmosphere and Surface Energy Balance',               fontsize=14, fontweight='bold')ax6.grid(True, axis='y', alpha=0.3)ax6.set_ylim(-400, 400)plt.suptitle('One-Dimensional Radiative-Convective Model: Complete Analysis',             fontsize=17, fontweight='bold', y=0.995)plt.savefig('model2_complete.png', dpi=150, bbox_inches='tight')plt.show()print("\n" + "="*70)print("KEY INSIGHTS FROM MODEL 2")print("="*70)print(f"\n1. Vertical Structure: Surface warmer ({T_control[0]-273.15:.1f}°C) than")print(f"   upper atmosphere ({T_control[-1]-273.15:.1f}°C) due to greenhouse effect")print(f"\n2. Climate Sensitivity: {ECS:.2f} K - closer to observations than Model 1")print("   due to inclusion of lapse rate and water vapor feedbacks")print(f"\n3. Stratospheric Cooling: Upper atmosphere cools with CO₂ increase")print("   while surface warms - characteristic signature of greenhouse forcing")print("\n4. Convective Control: Tropospheric lapse rate maintained near")print(f"   critical value ({model2.critical_lapse*1000:.1f} K/km) by convection")print("\n5. Greenhouse Back-radiation: Downward LW at surface")print(f"   ({F_down_sfc:.1f} W/m²) >> direct solar, demonstrating atmospheric effect")print("\n" + "="*70)

<a id='model3'></a>## Model 3: Two-Dimensional Statistical Dynamical Model### Technical Overview (Page 1 of 2)The Two-Dimensional Statistical Dynamical Model extends our framework by adding **latitudinal variation** while maintaining zonal (longitudinal) averaging. This captures the fundamental feature of Earth's climate: the equator-to-pole temperature gradient driven by differential solar heating.#### Governing EquationsThe model solves coupled equations for temperature and energy transport:**Thermodynamic Equation:**$$\rho c_p \frac{\partial T}{\partial t} = -\nabla \cdot \mathbf{F} + Q_{rad} + Q_{conv}$$**Meridional Energy Transport:**$$\mathbf{F} = -K \nabla T$$**Radiative Balance:**$$Q_{rad} = Q_{solar}(\phi) - \epsilon \sigma T^4$$Where:- $T(\phi, z, t)$ = Temperature as function of latitude $\phi$, height $z$, time $t$- $\mathbf{F}$ = Energy flux vector (atmosphere + ocean) [W m⁻²]- $K$ = Diffusion coefficient representing heat transport [W m⁻¹ K⁻¹]- $Q_{solar}(\phi) = \frac{S_0}{4}(1-\alpha)Q_{dist}(\phi)$ = Latitude-dependent solar heating- $Q_{dist}(\phi)$ = Distribution function (higher at equator, lower at poles)#### Key Physical Assumptions1. **Zonal Symmetry**: All variables are averaged in the longitudinal direction. No distinction between continents and oceans at same latitude.2. **Diffusive Heat Transport**: Complex atmospheric and oceanic dynamics (Hadley cells, jet streams, ocean gyres) parameterized as downgradient diffusion $F = -K\nabla T$. Real transport includes:   - Atmospheric: Baroclinic eddies, Hadley cell, Walker circulation   - Oceanic: Gyres, meridional overturning circulation, eddies   3. **Spherical Geometry**: Latitude-dependent area weighting:   $$\nabla \cdot \mathbf{F} = \frac{1}{R\cos\phi} \frac{\partial}{\partial \phi}(\cos\phi \cdot F_\phi)$$   where $R$ is Earth's radius.4. **Solar Distribution**: Incoming solar radiation depends on latitude:   $$Q_{solar}(\phi) \propto \cos\phi \text{ (approximately)}$$   More accurate: accounts for Earth's tilt and seasonal cycle (annual mean here).5. **Ice-Albedo Feedback**: Albedo $\alpha(\phi, T)$ increases when temperature drops below freezing:   $$\alpha = \begin{cases}   \alpha_{ocean} & T > 273 K \\   \alpha_{ice} & T < 273 K   \end{cases}$$   This creates positive feedback: cooling → more ice → higher albedo → more cooling.6. **Energy Balance Model (EBM) Form**: Often simplified to 1D in latitude:   $$C \frac{\partial T}{\partial t} = Q_{in}(\phi)(1-\alpha) - A - BT + \frac{1}{R^2\cos\phi}\frac{\partial}{\partial\phi}\left(\cos\phi \cdot D\frac{\partial T}{\partial\phi}\right)$$### Technical Overview (Page 2 of 2)#### Mathematical Approximations**Diffusive Transport Parameterization**: Real meridional energy transport is accomplished by:- **Atmospheric**:   - Hadley cell (tropical): Direct thermal circulation, ~100 PW  - Mid-latitude eddies: Baroclinic waves, ~50 PW    - Stationary waves: Mountain/heating contrasts- **Oceanic**:  - Wind-driven gyres: Gulf Stream, Kuroshio  - Thermohaline circulation: Atlantic MOC, ~1-2 PW  - Mesoscale eddiesDiffusion approximation:$$F = -K \frac{\partial T}{\partial \phi}$$where $K \approx 0.4-0.6$ W m⁻² K⁻¹ is calibrated to match observed transport (~6 PW from equator to pole). This is accurate for:- ✓ Time-mean transport- ✓ Large-scale patterns- ✗ Transient eddies- ✗ Non-local transport- ✗ Asymmetries between hemispheres**Linearized Outgoing Radiation**: Instead of $\epsilon\sigma T^4$, often use:$$OLR = A + BT$$where $A \approx 202$ W m⁻² and $B \approx 2.17$ W m⁻² K⁻¹ are fitted to match current climate. This is accurate for small perturbations ($\pm 10$ K) but breaks down for large changes.**Ice-Albedo Feedback**:Simple threshold:$$\alpha(\phi) = \begin{cases}0.32 & T > 273K \\0.62 & T < 273K  \end{cases}$$Reality is more complex:- Gradual transition via sea ice concentration- Snow on land vs sea ice- Seasonal cycle (summer melt, winter formation)- Multi-year ice vs first-year ice- Ice thickness and age effects**Solar Distribution**:Annual mean insolation at latitude $\phi$:$$Q(\phi) = \frac{S_0}{\pi}\left[H(\phi)\sin\phi\sin\delta + \cos\phi\cos\delta\sin H(\phi)\right]$$where $\delta$ is solar declination and $H$ is hour angle. For Earth:$$Q(\phi) \approx Q_0(1 + 0.482P_2(\sin\phi))$$where $P_2$ is Legendre polynomial. Common simplification:$$Q(\phi) = Q_0\left(1 - 0.482\left(\frac{3\sin^2\phi - 1}{2}\right)\right)$$#### Multiple Equilibria and BifurcationsA remarkable feature of 2D EBMs: **multiple equilibrium states**For current solar constant:1. **Warm climate** (current): Polar ice caps at ~70° latitude2. **Snowball Earth**: Global ice coverage (albedo catastrophe)3. **Ice-free**: No permanent ice (hothouse)Ice-albedo feedback creates **hysteresis**:- Decreasing $S_0$: Climate remains warm until critical point, then sudden transition to snowball- Increasing $S_0$: Snowball persists past the point where warm climate originally frozeCritical solar constant for snowball initiation: $S_c \approx 0.94 S_0$ (~6% reduction)#### Climate Sensitivity in 2D ModelsECS ≈ 2.5-3.5°C, higher than 1D models because:- ✓ Ice-albedo feedback included- ✓ Polar amplification captured: Arctic warms 2-3× faster than global mean- ✓ Pattern effects: Regional forcing distributions matter#### Limitations✗ **No longitudinal structure**: Cannot represent monsoons, ENSO, NAO✗ **No ocean dynamics**: Thermohaline circulation not resolved✗ **Simplified clouds**: Major uncertainty✗ **No topography**: Mountains affect circulation patterns✗ **Annual mean**: Seasonal cycle important for ice#### Applications✓ Paleoclimate: Snowball Earth, ice ages, Eocene hothouse✓ Conceptual understanding: Feedbacks, multiple equilibria✓ Computational efficiency: Fast scenario testing✓ Polar amplification: Captures Arctic warming pattern

In [ ]:
class TwoDimensionalEBM:    """    Two-Dimensional Energy Balance Model (latitude-height)        Includes meridional heat transport and ice-albedo feedback.    Demonstrates polar amplification and potential for multiple equilibria.    """        def __init__(self, n_lat=36, n_levels=10):        """        Initialize 2D EBM                Parameters:        -----------        n_lat : int            Number of latitude bands        n_levels : int            Number of vertical levels (simplified from Model 2)        """        # Physical constants        self.sigma = 5.67e-8    # Stefan-Boltzmann constant        self.S0 = 1361.0        # Solar constant        self.R_earth = 6.371e6  # Earth radius (m)                # Grid        self.n_lat = n_lat        self.lat = np.linspace(-90, 90, n_lat)  # Latitude (degrees)        self.lat_rad = np.deg2rad(self.lat)     # Latitude (radians)        self.d_lat = np.deg2rad(180 / (n_lat - 1))  # Grid spacing                # Model parameters        self.A = 202.0          # OLR parameter A (W/m²)        self.B = 2.17           # OLR parameter B (W/m²/K)        self.D = 0.44           # Diffusion coefficient (W/m²/K)        self.C = 4e7            # Heat capacity (J/m²/K) - mixed layer ocean        self.alpha_ocean = 0.32 # Ocean/land albedo        self.alpha_ice = 0.62   # Ice/snow albedo        self.T_freeze = 273.15  # Freezing temperature (K)            def solar_distribution(self, S0=None):        """        Calculate latitude-dependent solar input                Uses 2nd Legendre polynomial for annual mean insolation                Returns:        --------        Q : array            Solar input at each latitude (W/m²)        """        if S0 is None:            S0 = self.S0                Q0 = S0 / 4  # Global mean                # Legendre P2 distribution        sin_lat = np.sin(self.lat_rad)        P2 = (3 * sin_lat**2 - 1) / 2                Q = Q0 * (1 - 0.482 * P2)                return Q        def albedo(self, T):        """        Calculate albedo with ice-albedo feedback                Parameters:        -----------        T : array            Temperature at each latitude (K)                    Returns:        --------        alpha : array            Albedo at each latitude        """        alpha = np.where(T < self.T_freeze, self.alpha_ice, self.alpha_ocean)        return alpha        def outgoing_longwave(self, T):        """        Outgoing longwave radiation (linearized)                OLR = A + B*T        """        return self.A + self.B * T        def absorbed_solar(self, T, S0=None):        """        Absorbed solar radiation including albedo feedback        """        Q = self.solar_distribution(S0)        alpha = self.albedo(T)        return Q * (1 - alpha)        def diffusion_operator(self, T):        """        Compute meridional heat transport via diffusion                ∇·F = (1/R²cos(φ)) ∂/∂φ [cos(φ) D ∂T/∂φ]                Returns:        --------        div_F : array            Divergence of heat flux (W/m²)        """        # Compute temperature gradient        dT_dlat = np.gradient(T, self.d_lat)                # Compute flux with cos(φ) weighting        cos_lat = np.cos(self.lat_rad)        flux = -self.D * cos_lat * dT_dlat                # Compute divergence        dflux_dlat = np.gradient(flux, self.d_lat)        div_F = dflux_dlat / (self.R_earth * cos_lat)                return div_F        def tendency(self, T, S0=None):        """        Calculate temperature tendency dT/dt                C dT/dt = Q(1-α) - (A+BT) + ∇·F                Returns:        --------        dT_dt : array              Temperature tendency (K/s)        """        Q_abs = self.absorbed_solar(T, S0)        OLR = self.outgoing_longwave(T)        div_F = self.diffusion_operator(T)                # Net heating        Q_net = Q_abs - OLR + div_F                # Convert to temperature tendency        dT_dt = Q_net / self.C                return dT_dt        def run_to_equilibrium(self, T_init=None, years=100, dt=0.1, S0=None):        """        Time-step to equilibrium                Parameters:        -----------        T_init : array, optional            Initial temperature profile (K)        years : float            Integration time (years)        dt : float            Time step (years)        S0 : float, optional            Solar constant (W/m²), default is self.S0                    Returns:        --------        T : array            Final temperature profile (K)        T_history : array            Temperature evolution [time, lat]        """        # Initialize        if T_init is None:            # Reasonable initial guess            T = 288 - 40 * np.abs(np.sin(self.lat_rad))  # Warmer equator, colder poles        else:            T = T_init.copy()                # Time integration        seconds_per_year = 365.25 * 24 * 3600        dt_seconds = dt * seconds_per_year        n_steps = int(years / dt)                # Store some history        save_interval = max(1, n_steps // 200)        T_history = [T.copy()]        times = [0]                for step in range(n_steps):            # Forward Euler            dT_dt = self.tendency(T, S0)            T = T + dT_dt * dt_seconds                        # Save periodically            if step % save_interval == 0:                T_history.append(T.copy())                times.append(step * dt)                return T, np.array(T_history), np.array(times)        def find_ice_edge(self, T):        """        Find latitude of ice edge (freezing isotherm)                Returns:        --------        ice_edge_north : float            Northern hemisphere ice edge latitude (degrees)        ice_edge_south : float            Southern hemisphere ice edge latitude (degrees)        """        # Northern hemisphere        nh_idx = self.lat >= 0        T_nh = T[nh_idx]        lat_nh = self.lat[nh_idx]                if np.any(T_nh < self.T_freeze):            idx = np.where(T_nh < self.T_freeze)[0][0]            ice_edge_north = lat_nh[idx]        else:            ice_edge_north = 90  # No ice                # Southern hemisphere          sh_idx = self.lat <= 0        T_sh = T[sh_idx]        lat_sh = self.lat[sh_idx]                if np.any(T_sh < self.T_freeze):            idx = np.where(T_sh < self.T_freeze)[0][-1]            ice_edge_south = lat_sh[idx]        else:            ice_edge_south = -90  # No ice                return ice_edge_north, ice_edge_south        def climate_sensitivity(self, forcing=4.0):        """        Calculate ECS by running control and forced experiments                CO2 forcing applied as uniform heating        """        # Control        T_control, _, _ = self.run_to_equilibrium(years=50, dt=0.1)                # Forced (approximate CO2 forcing as reduced OLR)        # Equivalent to reducing A parameter        A_original = self.A        self.A = A_original - forcing                T_forced, _, _ = self.run_to_equilibrium(T_init=T_control, years=50, dt=0.1)                # Restore        self.A = A_original                # Calculate ECS (global mean)        ECS = np.mean(T_forced - T_control)                return T_control, T_forced, ECS# Initialize and run Model 3print("="*70)print("TWO-DIMENSIONAL ENERGY BALANCE MODEL")print("="*70 + "\n")model3 = TwoDimensionalEBM(n_lat=36)print(f"Model Configuration:")print(f"  Latitudes: {model3.n_lat} bands from {model3.lat[0]:.0f}° to {model3.lat[-1]:.0f}°")print(f"  Diffusion coefficient (D): {model3.D:.3f} W/m²/K")print(f"  Heat capacity (C): {model3.C:.2e} J/m²/K")print(f"  Albedo: {model3.alpha_ocean:.2f} (open) → {model3.alpha_ice:.2f} (ice)\n")print("Computing equilibrium climate...")T_eq_2d, T_history, times = model3.run_to_equilibrium(years=50, dt=0.1)ice_n, ice_s = model3.find_ice_edge(T_eq_2d)print(f"  Global mean temperature: {np.mean(T_eq_2d):.2f} K ({np.mean(T_eq_2d)-273.15:.2f}°C)")print(f"  Equatorial temperature: {T_eq_2d[model3.n_lat//2]:.2f} K ({T_eq_2d[model3.n_lat//2]-273.15:.2f}°C)")print(f"  Polar temperatures: {np.mean([T_eq_2d[0], T_eq_2d[-1]]):.2f} K ({np.mean([T_eq_2d[0], T_eq_2d[-1]])-273.15:.2f}°C)")print(f"  Ice edge: North {ice_n:.1f}°, South {ice_s:.1f}°\n")print("Computing climate sensitivity...")T_control_2d, T_forced_2d, ECS_2d = model3.climate_sensitivity(forcing=4.0)print(f"  Global mean ECS: {ECS_2d:.2f} K")print(f"  Equatorial ECS: {(T_forced_2d - T_control_2d)[model3.n_lat//2]:.2f} K")print(f"  Polar ECS: {np.mean([(T_forced_2d - T_control_2d)[0], (T_forced_2d - T_control_2d)[-1]]):.2f} K")print(f"  Polar amplification factor: {np.mean([(T_forced_2d - T_control_2d)[0], (T_forced_2d - T_control_2d)[-1]]) / ECS_2d:.2f}×")print("\n" + "="*70)

In [ ]:
# Visualize Model 3: Two-Dimensional Energy Balance Modelfig = plt.figure(figsize=(16, 12))gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)# === Panel 1: Temperature vs Latitude ===ax1 = fig.add_subplot(gs[0, 0])ax1.plot(model3.lat, T_control_2d - 273.15, 'b-', linewidth=3,          label='Control', marker='o', markersize=4)ax1.plot(model3.lat, T_forced_2d - 273.15, 'r-', linewidth=3,         label='2×CO₂', marker='s', markersize=4)ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)ax1.axhline(model3.T_freeze - 273.15, color='cyan', linestyle=':',             linewidth=2, alpha=0.7, label='Freezing point')ax1.set_xlabel('Latitude (°)', fontsize=12, fontweight='bold')ax1.set_ylabel('Temperature (°C)', fontsize=12, fontweight='bold')ax1.set_title('Temperature Distribution by Latitude', fontsize=14, fontweight='bold')ax1.legend(fontsize=11, loc='lower center')ax1.grid(True, alpha=0.3)ax1.set_xlim(-90, 90)# === Panel 2: Warming Pattern ===ax2 = fig.add_subplot(gs[0, 1])delta_T_2d = T_forced_2d - T_control_2dax2.plot(model3.lat, delta_T_2d, 'purple', linewidth=3, marker='d', markersize=5)ax2.axhline(ECS_2d, color='gray', linestyle='--', alpha=0.5, label=f'Global mean: {ECS_2d:.2f} K')ax2.fill_between(model3.lat, 0, delta_T_2d, alpha=0.3, color='red')ax2.set_xlabel('Latitude (°)', fontsize=12, fontweight='bold')ax2.set_ylabel('Temperature Change (K)', fontsize=12, fontweight='bold')ax2.set_title('Polar Amplification Pattern', fontsize=14, fontweight='bold')ax2.legend(fontsize=11)ax2.grid(True, alpha=0.3)ax2.set_xlim(-90, 90)# === Panel 3: Albedo Distribution ===ax3 = fig.add_subplot(gs[1, 0])albedo_control = model3.albedo(T_control_2d)albedo_forced = model3.albedo(T_forced_2d)ax3.plot(model3.lat, albedo_control, 'b-', linewidth=3, label='Control')ax3.plot(model3.lat, albedo_forced, 'r--', linewidth=3, label='2×CO₂')ax3.fill_between(model3.lat, albedo_control, albedo_forced,                  where=(albedo_forced < albedo_control),                   alpha=0.3, color='red', label='Ice retreat')ax3.set_xlabel('Latitude (°)', fontsize=12, fontweight='bold')ax3.set_ylabel('Albedo', fontsize=12, fontweight='bold')ax3.set_title('Ice-Albedo Feedback', fontsize=14, fontweight='bold')ax3.legend(fontsize=11)ax3.grid(True, alpha=0.3)ax3.set_xlim(-90, 90)ax3.set_ylim(0.2, 0.7)# === Panel 4: Energy Fluxes ===ax4 = fig.add_subplot(gs[1, 1])Q_solar = model3.solar_distribution()Q_absorbed = Q_solar * (1 - albedo_control)OLR = model3.outgoing_longwave(T_control_2d)ax4.plot(model3.lat, Q_solar, 'orange', linewidth=2.5, label='Incident Solar')ax4.plot(model3.lat, Q_absorbed, 'gold', linewidth=2.5, label='Absorbed Solar')ax4.plot(model3.lat, OLR, 'blue', linewidth=2.5, label='Outgoing LW')ax4.plot(model3.lat, Q_absorbed - OLR, 'green', linewidth=2.5,          linestyle='--', label='Net (requires transport)')ax4.axhline(0, color='gray', linestyle='--', alpha=0.5)ax4.set_xlabel('Latitude (°)', fontsize=12, fontweight='bold')ax4.set_ylabel('Energy Flux (W/m²)', fontsize=12, fontweight='bold')ax4.set_title('Radiative Balance by Latitude', fontsize=14, fontweight='bold')ax4.legend(fontsize=10, loc='upper right')ax4.grid(True, alpha=0.3)ax4.set_xlim(-90, 90)# === Panel 5: Meridional Heat Transport ===ax5 = fig.add_subplot(gs[2, :])# Calculate transport (integral of imbalance)imbalance = Q_absorbed - OLRcos_lat = np.cos(model3.lat_rad)# Integrate from south to northtransport = np.zeros_like(imbalance)for i in range(1, len(imbalance)):    # Transport = integral of imbalance weighted by cos(lat)    transport[i] = transport[i-1] + (imbalance[i-1] + imbalance[i])/2 * \                   (cos_lat[i-1] + cos_lat[i])/2 * model3.d_lat * model3.R_earth# Convert to Petawattstransport_PW = transport * 2 * np.pi * model3.R_earth / 1e15ax5.plot(model3.lat, transport_PW, 'darkgreen', linewidth=4)ax5.axhline(0, color='gray', linestyle='--', alpha=0.5)ax5.axvline(0, color='gray', linestyle='--', alpha=0.5)ax5.fill_between(model3.lat, 0, transport_PW, where=(transport_PW > 0),                  alpha=0.3, color='red', label='Northward')ax5.fill_between(model3.lat, 0, transport_PW, where=(transport_PW < 0),                  alpha=0.3, color='blue', label='Southward')ax5.set_xlabel('Latitude (°)', fontsize=13, fontweight='bold')ax5.set_ylabel('Meridional Heat Transport (PW)', fontsize=13, fontweight='bold')ax5.set_title('Poleward Energy Transport (Atmosphere + Ocean)',               fontsize=15, fontweight='bold')ax5.legend(fontsize=11)ax5.grid(True, alpha=0.3)ax5.set_xlim(-90, 90)max_transport = np.max(np.abs(transport_PW))print(f"  Maximum poleward transport: {max_transport:.2f} PW")plt.suptitle('Two-Dimensional Energy Balance Model: Meridional Structure',             fontsize=17, fontweight='bold', y=0.995)plt.savefig('model3_complete.png', dpi=150, bbox_inches='tight')plt.show()print("\n" + "="*70)print("KEY INSIGHTS FROM MODEL 3")print("="*70)print("\n1. Equator-Pole Gradient: Temperature decreases from equator to poles")print(f"   Equator: {T_control_2d[model3.n_lat//2]-273.15:.1f}°C, Poles: {np.mean([T_control_2d[0], T_control_2d[-1]])-273.15:.1f}°C")print(f"\n2. Polar Amplification: Arctic/Antarctic warm {np.mean([(T_forced_2d - T_control_2d)[0], (T_forced_2d - T_control_2d)[-1]])/ECS_2d:.1f}× faster")print("   than global mean due to ice-albedo feedback")print(f"\n3. Ice-Albedo Feedback: Positive feedback as ice retreat")print("   lowers albedo, causing more warming")print(f"\n4. Meridional Transport: ~{max_transport:.1f} PW transported from")print("   tropics to poles by atmosphere and ocean")print("\n5. Energy Imbalance: Tropics have surplus, poles have deficit")print("   → drives atmospheric/oceanic circulation")print("\n" + "="*70)

<a id='model4'></a>## Model 4: Three-Dimensional General Circulation Model (GCM)### Technical Overview (Page 1 of 2)Three-Dimensional General Circulation Models represent the state-of-the-art in traditional climate modeling. These models explicitly resolve atmospheric and oceanic circulation in three spatial dimensions and time, governed by the fundamental equations of fluid dynamics and thermodynamics.#### Governing EquationsGCMs solve the **primitive equations** on a 3D grid:**1. Momentum (Navier-Stokes):**$$\frac{D\mathbf{u}}{Dt} + 2\mathbf{\Omega} \times \mathbf{u} = -\frac{1}{\rho}\nabla p + \mathbf{g} + \mathbf{F}$$**2. Continuity (Mass Conservation):**$$\frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \mathbf{u}) = 0$$**3. Thermodynamic Energy:**$$\rho c_p \frac{DT}{Dt} = \frac{Dp}{Dt} + Q_{rad} + Q_{latent} + Q_{sens}$$**4. Water Vapor:**$$\frac{Dq}{Dt} = S_{evap} - S_{precip} + \text{diffusion}$$**5. Hydrostatic Balance (vertical):**$$\frac{\partial p}{\partial z} = -\rho g$$Where:- $\mathbf{u} = (u, v, w)$ = 3D velocity field (m/s)- $\mathbf{\Omega}$ = Earth's rotation vector- $\rho$ = Air/water density (kg/m³)- $p$ = Pressure (Pa)- $T$ = Temperature (K)- $q$ = Specific humidity (kg/kg)- $Q$ = Heating/cooling terms (W/kg)#### Model Components**Atmospheric Model:**- Horizontal resolution: 50-200 km (lat-lon grid or spectral)- Vertical levels: 30-100 (surface to ~50-100 km)- Time step: 10-30 minutes- Prognostic variables: u, v, w, T, p, q, clouds**Ocean Model:**- Resolution: 25-100 km horizontal, 40-60 vertical levels- Dynamics: Full 3D primitive equations- Tracers: Temperature, salinity, biogeochemistry- Sea ice: Thermodynamics and dynamics**Land Surface Model:**- Soil moisture, temperature (multiple layers)- Vegetation: Types, phenology, photosynthesis- Snow cover and albedo- Runoff and groundwater**Cryosphere:**- Sea ice: Thickness, concentration, dynamics- Land ice: Mass balance (simple) or ice sheet model (advanced)- Snow cover: Depth, density, albedo evolution#### Physical Parameterizations**Sub-grid processes** that cannot be resolved explicitly:1. **Radiation**:    - Solar: Rayleigh scattering, absorption by O₃, H₂O, clouds   - Longwave: Line-by-line or band models for greenhouse gases   - Computed every 1-3 hours (expensive!)2. **Convection**:   - Deep convection (thunderstorms): Mass flux schemes   - Shallow convection: Eddy diffusivity   - Triggers: CAPE, moisture convergence   - Outputs: Precipitation, heating/moistening profiles3. **Clouds**:   - Formation: Relative humidity threshold or PDF-based   - Types: Stratiform vs convective   - Microphysics: Condensation, freezing, precipitation   - Huge uncertainty source!4. **Boundary Layer Turbulence**:   - Vertical mixing of heat, moisture, momentum   - K-theory, TKE schemes, or higher-order closure   - Surface fluxes: Bulk aerodynamic formulae5. **Gravity Wave Drag**:   - Orographic: Mountain effects on flow   - Non-orographic: Convection, fronts   - Critical for stratospheric circulation### Technical Overview (Page 2 of 2)#### Numerical Methods**Spatial Discretization:**- **Finite Difference**: Grid points, simple but diffusive- **Finite Volume**: Conservative, good for tracers- **Spectral**: Spherical harmonics, accurate but expensive- **Finite Element**: Flexible grids (icosahedral)**Temporal Integration:**- **Semi-implicit**: Large time steps for stable modes- **Split-explicit**: Fast/slow modes separated- **Leapfrog, RK schemes**: Various orders of accuracy**Grids:**- Lat-lon: Simple but pole singularity- Cubed-sphere: 6 patches, more uniform- Icosahedral: Triangular cells, nearly uniform- Variable resolution: Regional refinement#### Key Approximations1. **Hydrostatic Approximation**:    $$\frac{\partial p}{\partial z} = -\rho g$$   Valid for horizontal scales >> vertical scale (~10 km)   Breaks down for deep convection, topography2. **Boussinesq Approximation**:   Density variations neglected except in buoyancy   Valid for small density variations3. **Shallow Atmosphere**:   Earth's radius >> atmospheric depth   Metric terms simplified4. **Sub-grid Parameterizations**:   Most critical approximation! Clouds, convection, turbulence   cannot be resolved and must be parameterized   → Largest uncertainty in GCMs5. **Resolution Limits**:   - Cannot resolve individual clouds (km scale)   - Cannot resolve ocean mesoscale eddies (<50 km)   - Cannot resolve boundary layer turbulence (m scale)   #### Climate Sensitivity in GCMsModern GCMs: ECS = 2.5-5.0°C (IPCC AR6 range: 2.5-4.0°C likely)Higher sensitivity than simpler models due to:- ✓ Cloud feedbacks (most uncertain!)- ✓ Water vapor feedback (well-constrained)- ✓ Ice-albedo feedback- ✓ Lapse rate feedback- ✓ Regional patterns and teleconnections**Feedback Analysis:**$$\text{ECS} = \frac{\lambda_0}{1 - \sum f_i}$$where $f_i$ are individual feedbacks:- $f_{H_2O} ≈ +0.5$ (strongly positive)- $f_{ice} ≈ +0.3$ (positive)- $f_{cloud} ≈ +0.2$ to +0.8 (uncertain!)- $f_{lapse} ≈ -0.2$ (negative)#### Advantages Over Simpler Models✓ Explicit dynamics: Jets, storms, monsoons, ENSO✓ Regional detail: Precipitation, drought, extremes✓ Coupled system: Ocean-atmosphere interactions✓ Tracers: CO₂, aerosols, chemistry✓ Transient response: Decades to centuries✓ Multiple forcings: GHGs, aerosols, land use#### Limitations✗ Computationally expensive: Weeks to months for century runs✗ Parameterization uncertainty: Sub-grid physics✗ Systematic biases: Regional temperature/precipitation errors✗ Limited resolution: Cannot resolve small scales✗ Initialization: Sensitive to initial conditions (weather scales)#### ValidationGCMs are validated against:- Historical climate (1850-present)- Paleoclimate (Last Glacial Maximum, Mid-Holocene)- Satellite observations (radiation, temperature, clouds)- Reanalysis data- Process studies**Key Metrics:**- Mean state climatology- Seasonal cycle- Interannual variability (ENSO, NAO)- Trends (warming, sea level rise)- Extreme events

In [ ]:
# Model 4: Simplified 3D General Circulation Model# # Full GCMs are too complex to implement from scratch here.# Instead, we demonstrate the conceptual framework and show # representative outputs from a simplified 3-cell circulation model.class SimplifiedGCM:    """    Highly simplified GCM demonstrating key concepts:    - 3D grid structure (coarse resolution)    - Hadley cell circulation    - Temperature and pressure fields    - Simplified dynamics        This is NOT a full GCM but demonstrates the principles.    """        def __init__(self, n_lat=18, n_lon=36, n_lev=10):        """Initialize simplified GCM grid"""        self.n_lat = n_lat        self.n_lon = n_lon        self.n_lev = n_lev                # Create grid        self.lat = np.linspace(-90, 90, n_lat)        self.lon = np.linspace(0, 360, n_lon, endpoint=False)        self.lev = np.linspace(1000, 100, n_lev)  # Pressure levels (hPa)                # Physical parameters        self.omega = 7.292e-5  # Earth rotation (rad/s)        self.R_earth = 6.371e6  # Earth radius (m)        self.g = 9.81        self.R = 287.0  # Gas constant            def initialize_fields(self):        """        Initialize temperature and wind fields        Based on observed climatology        """        # Create 3D grids        LAT, LON, LEV = np.meshgrid(self.lat, self.lon, self.lev, indexing='ij')                # Temperature: Decreases poleward and with height        lat_rad = np.deg2rad(LAT)        T = 300 - 50*np.abs(np.sin(lat_rad)) - 50*(1000 - LEV)/1000                # Zonal wind (westerlies): Peaks at mid-latitudes        u = 20 * np.sin(2*lat_rad) * (LEV/500)**0.5                # Meridional circulation (Hadley cell): Simplified        v = -10 * np.sin(lat_rad) * np.cos(lat_rad) * (LEV/1000)                # Vertical velocity (pressure coordinates): Small        omega = 0.01 * np.sin(2*lat_rad)  # Pa/s                return {            'T': T,            'u': u,            'v': v,            'omega': omega,            'lat': LAT,            'lon': LON,            'lev': LEV        }        def compute_circulation_strength(self, fields):        """Compute meridional overturning circulation"""        # Mass streamfunction (simplified)        v = fields['v']                # Zonally average        v_zonal = np.mean(v, axis=1)                # Integrate vertically        psi = np.zeros_like(v_zonal)        for i in range(1, self.n_lev):            psi[:, i] = psi[:, i-1] + v_zonal[:, i] * 100  # Rough integration                return psi        def apply_greenhouse_forcing(self, fields, forcing=4.0):        """        Simulate warming response to GHG forcing                Simple parameterization:        - Uniform forcing applied        - Polar amplification factor        - Stratospheric cooling        """        T = fields['T'].copy()        lat_rad = np.deg2rad(fields['lat'])        lev = fields['lev']                # Surface warming with polar amplification        warming = forcing * 0.75  # ~3K for 4 W/m²        polar_amp = 1 + 1.5*np.abs(np.sin(lat_rad))                # Height dependence: warming at surface, cooling aloft        height_factor = np.where(lev > 300, 1.0, 1.0 - 0.5*(lev/300-1))        height_factor = np.maximum(height_factor, -0.5)                T_new = T + warming * polar_amp * height_factor                return T_new# Initialize simplified GCMprint("="*70)print("THREE-DIMENSIONAL GENERAL CIRCULATION MODEL (Simplified)")print("="*70 + "\n")gcm = SimplifiedGCM(n_lat=18, n_lon=36, n_lev=10)print(f"Model Configuration:")print(f"  Grid: {gcm.n_lat}°×{gcm.n_lon}° × {gcm.n_lev} levels")print(f"  Resolution: {180/gcm.n_lat:.1f}° lat × {360/gcm.n_lon:.1f}° lon")print(f"  Pressure range: {gcm.lev[-1]:.0f} - {gcm.lev[0]:.0f} hPa\n")print("Initializing atmospheric fields from climatology...")fields_control = gcm.initialize_fields()print(f"  Surface temperature range: {np.min(fields_control['T'][:,:,0]):.1f} - {np.max(fields_control['T'][:,:,0]):.1f} K")print(f"  Maximum zonal wind: {np.max(fields_control['u']):.1f} m/s")print(f"  Meridional circulation: Hadley cells present\n")print("Computing circulation patterns...")psi_control = gcm.compute_circulation_strength(fields_control)print("Applying greenhouse forcing (+4 W/m²)...")fields_forced = fields_control.copy()fields_forced['T'] = gcm.apply_greenhouse_forcing(fields_control, forcing=4.0)delta_T_gcm = fields_forced['T'] - fields_control['T']global_mean_warming = np.mean(delta_T_gcm[:,:,0])print(f"  Global mean surface warming: {global_mean_warming:.2f} K")print(f"  Polar warming: {np.mean([np.mean(delta_T_gcm[0,:,0]), np.mean(delta_T_gcm[-1,:,0])]):.2f} K")print(f"  Tropical warming: {np.mean(delta_T_gcm[gcm.n_lat//2-2:gcm.n_lat//2+2,:,0]):.2f} K")print("\n" + "="*70)

In [ ]:
# Visualize Model 4: 3D GCM Fieldsfig = plt.figure(figsize=(18, 14))# Extract surface fieldsT_sfc_control = fields_control['T'][:,:,0]T_sfc_forced = fields_forced['T'][:,:,0]u_200hPa = fields_control['u'][:,:,-3]  # Upper troposphere# === Panel 1: Surface Temperature (Control) ===ax1 = plt.subplot(3, 3, 1)LON_grid, LAT_grid = np.meshgrid(gcm.lon, gcm.lat)c1 = ax1.contourf(LON_grid, LAT_grid, T_sfc_control-273.15,                   levels=20, cmap='RdBu_r')ax1.contour(LON_grid, LAT_grid, T_sfc_control-273.15, levels=10,             colors='black', alpha=0.3, linewidths=0.5)plt.colorbar(c1, ax=ax1, label='Temperature (°C)')ax1.set_xlabel('Longitude (°)')ax1.set_ylabel('Latitude (°)')ax1.set_title('Control: Surface Temperature', fontweight='bold')# === Panel 2: Surface Temperature (2×CO₂) ===ax2 = plt.subplot(3, 3, 2)c2 = ax2.contourf(LON_grid, LAT_grid, T_sfc_forced-273.15,                  levels=20, cmap='RdBu_r')ax2.contour(LON_grid, LAT_grid, T_sfc_forced-273.15, levels=10,            colors='black', alpha=0.3, linewidths=0.5)plt.colorbar(c2, ax=ax2, label='Temperature (°C)')ax2.set_xlabel('Longitude (°)')ax2.set_ylabel('Latitude (°)')ax2.set_title('2×CO₂: Surface Temperature', fontweight='bold')# === Panel 3: Temperature Change ===ax3 = plt.subplot(3, 3, 3)delta_T_sfc = T_sfc_forced - T_sfc_controlc3 = ax3.contourf(LON_grid, LAT_grid, delta_T_sfc,                  levels=15, cmap='Reds')ax3.contour(LON_grid, LAT_grid, delta_T_sfc, levels=8,            colors='black', alpha=0.4, linewidths=0.5)plt.colorbar(c3, ax=ax3, label='ΔT (K)')ax3.set_xlabel('Longitude (°)')ax3.set_ylabel('Latitude (°)')ax3.set_title('Surface Warming Pattern', fontweight='bold')# === Panel 4: Upper-Level Winds ===ax4 = plt.subplot(3, 3, 4)c4 = ax4.contourf(LON_grid, LAT_grid, u_200hPa, levels=20, cmap='RdBu_r')ax4.contour(LON_grid, LAT_grid, u_200hPa, levels=10,            colors='black', alpha=0.3, linewidths=0.5)plt.colorbar(c4, ax=ax4, label='Zonal Wind (m/s)')ax4.set_xlabel('Longitude (°)')ax4.set_ylabel('Latitude (°)')ax4.set_title('Upper Troposphere Zonal Wind (200 hPa)', fontweight='bold')# === Panel 5: Vertical Temperature Profile ===ax5 = plt.subplot(3, 3, 5)# Extract tropical and polar profileslat_tropical_idx = gcm.n_lat // 2lat_polar_idx = -1T_tropical = np.mean(fields_control['T'][lat_tropical_idx, :, :], axis=0)T_polar = np.mean(fields_control['T'][lat_polar_idx, :, :], axis=0)ax5.plot(T_tropical-273.15, gcm.lev, 'r-', linewidth=3, marker='o', label='Tropical')ax5.plot(T_polar-273.15, gcm.lev, 'b-', linewidth=3, marker='s', label='Polar')ax5.invert_yaxis()ax5.set_yscale('log')ax5.set_xlabel('Temperature (°C)', fontweight='bold')ax5.set_ylabel('Pressure (hPa)', fontweight='bold')ax5.set_title('Vertical Temperature Profiles', fontweight='bold')ax5.legend()ax5.grid(True, alpha=0.3)# === Panel 6: Meridional Overturning ===ax6 = plt.subplot(3, 3, 6)c6 = ax6.contourf(gcm.lat, gcm.lev, psi_control.T, levels=15, cmap='RdBu_r')plt.colorbar(c6, ax=ax6, label='Streamfunction')ax6.invert_yaxis()ax6.set_xlabel('Latitude (°)', fontweight='bold')ax6.set_ylabel('Pressure (hPa)', fontweight='bold')ax6.set_title('Meridional Overturning Circulation', fontweight='bold')ax6.axvline(0, color='black', linestyle='--', alpha=0.3)# === Panel 7: Zonal Mean Temperature (Control) ===ax7 = plt.subplot(3, 3, 7)T_zonal_control = np.mean(fields_control['T'], axis=1)c7 = ax7.contourf(gcm.lat, gcm.lev, T_zonal_control.T-273.15,                  levels=20, cmap='RdBu_r')plt.colorbar(c7, ax=ax7, label='Temperature (°C)')ax7.invert_yaxis()ax7.set_xlabel('Latitude (°)', fontweight='bold')ax7.set_ylabel('Pressure (hPa)', fontweight='bold')ax7.set_title('Zonal Mean Temperature (Control)', fontweight='bold')# === Panel 8: Zonal Mean Temperature (2×CO₂) ===ax8 = plt.subplot(3, 3, 8)T_zonal_forced = np.mean(fields_forced['T'], axis=1)c8 = ax8.contourf(gcm.lat, gcm.lev, T_zonal_forced.T-273.15,                  levels=20, cmap='RdBu_r')plt.colorbar(c8, ax=ax8, label='Temperature (°C)')ax8.invert_yaxis()ax8.set_xlabel('Latitude (°)', fontweight='bold')ax8.set_ylabel('Pressure (hPa)', fontweight='bold')ax8.set_title('Zonal Mean Temperature (2×CO₂)', fontweight='bold')# === Panel 9: Warming by Height and Latitude ===ax9 = plt.subplot(3, 3, 9)delta_T_zonal = T_zonal_forced - T_zonal_controlc9 = ax9.contourf(gcm.lat, gcm.lev, delta_T_zonal.T,                  levels=15, cmap='Reds')ax9.contour(gcm.lat, gcm.lev, delta_T_zonal.T, levels=8,            colors='black', alpha=0.4, linewidths=0.5)plt.colorbar(c9, ax=ax9, label='ΔT (K)')ax9.invert_yaxis()ax9.set_xlabel('Latitude (°)', fontweight='bold')ax9.set_ylabel('Pressure (hPa)', fontweight='bold')ax9.set_title('Warming Pattern (Height×Latitude)', fontweight='bold')plt.suptitle('Three-Dimensional General Circulation Model: Global Fields',             fontsize=18, fontweight='bold', y=0.995)plt.tight_layout()plt.savefig('model4_complete.png', dpi=150, bbox_inches='tight')plt.show()print("\n" + "="*70)print("KEY INSIGHTS FROM MODEL 4 (Simplified GCM)")print("="*70)print("\n1. 3D Structure: Temperature, winds, and circulation vary in all")print("   three spatial dimensions - latitude, longitude, height")print(f"\n2. Atmospheric Dynamics: Jet streams at mid-latitudes,")print("   Hadley cells in tropics explicitly represented")print(f"\n3. Global Mean Warming: {global_mean_warming:.2f} K with strong")print("   polar amplification pattern")print("\n4. Vertical Structure: Surface warming, stratospheric cooling")print("   characteristic of greenhouse forcing")print("\n5. Full GCMs: Real climate models have:")print("   - Much higher resolution (50-100 km)")print("   - Coupled ocean model")print("   - Full radiative transfer")print("   - Cloud microphysics")print("   - Land surface and ice sheet models")print("   - Run for decades to centuries of simulated time")print("\n" + "="*70)

<a id='model5'></a>## Model 5: GraphCast - ML-Based Weather and Climate Prediction### Technical Overview (Page 1 of 2)GraphCast, developed by Google DeepMind, represents a paradigm shift in weather and climate modeling. Instead of explicitly solving physical equations, it uses machine learning to learn patterns from historical data and make predictions. This approach achieves competitive or superior accuracy to traditional physics-based models while being orders of magnitude faster.#### Architecture and Approach**Core Innovation:**GraphCast uses a **Graph Neural Network (GNN)** operating on a multi-resolution mesh of Earth's surface and atmosphere. Unlike traditional grid-based models, the graph structure allows flexible representation of Earth's spherical geometry and multi-scale processes.**Model Architecture:**1. **Input Representation:**   - Two atmospheric states: current time $t$ and $t-\Delta t$   - Variables: Temperature, winds (u,v), pressure, humidity, geopotential at multiple levels   - Surface variables: Temperature, pressure, moisture   - Grid: ~0.25° resolution (~28 km at equator), 37 pressure levels2. **Encoder:**   - Maps gridded data to graph representation   - Each grid point → graph node   - Edges connect nearby nodes (multi-resolution)   3. **Processor:**   - 16 layers of message-passing GNN   - Each layer: nodes aggregate information from neighbors   - Attention mechanisms weight importance   - ~37 million parameters total   4. **Decoder:**   - Maps graph back to grid   - Outputs: Future state at $t+\Delta t$ (typically 6 hours)   5. **Autoregressive Rollout:**   - Multi-step predictions: use output as input for next step   - 10-day forecast: 40 steps of 6-hour predictions#### Training Data and Process**Data:**- ERA5 reanalysis (ECMWF): 1979-2017 (training), 2018-2021 (validation/test)- ~1.4 million atmospheric states- All weather conditions: hurricanes, monsoons, heatwaves, etc.**Training:**- Loss function: Weighted MSE + gradient penalties- Emphasis on:  - Conservation of physical quantities  - Smooth spatial fields  - Realistic amplitudes and patterns  **Objective:**$$\mathcal{L} = \sum_{i,t} w_i ||X_{pred}^{t+\Delta t} - X_{true}^{t+\Delta t}||^2 + \lambda ||\nabla X_{pred}||^2$$where $w_i$ are pressure-dependent weights (emphasize troposphere).#### Key Physical Constraints (Learned, Not Enforced)Unlike traditional models that explicitly solve conservation laws, GraphCast learns to respect them through data:1. **Mass Conservation**: Total atmospheric mass should not change2. **Energy Conservation**: KE + PE + IE balanced3. **Geostrophic Balance**: Winds and pressure gradients related4. **Hydrostatic Balance**: Vertical pressure-temperature relationship5. **Water Cycle**: Evaporation ≈ Precipitation (global mean)These emerge from training, not hard constraints!#### Advantages of ML Approach✓ **Speed**: 1-minute runtime for 10-day forecast (vs hours for traditional GCMs)✓ **Scalability**: Inference cost independent of forecast length✓ **Data-driven**: Learns complex patterns humans cannot parameterize✓ **Resolution**: Fine-scale features without explicit sub-grid models✓ **Flexibility**: Easy to add new variables or change resolution#### Limitations✗ **Data-dependent**: Cannot predict beyond training distribution   - Novel climate states (e.g., 4°C warmer) uncertain   - Rare extremes underrepresented in training data   ✗ **Black box**: Difficult to interpret why predictions made✗ **Physical consistency**: May violate conservation laws subtly✗ **Long-term drift**: Accumulates errors over many time steps✗ **Extrapolation**: Struggles with unprecedented conditions### Technical Overview (Page 2 of 2)#### Comparison: GraphCast vs Traditional GCMs| Aspect | Traditional GCM | GraphCast ||--------|----------------|-----------|| **Physics** | Explicit equations | Learned from data || **Speed** | Hours (10-day forecast) | ~1 minute || **Resolution** | 25-100 km | ~25 km || **Accuracy** | Benchmark standard | Competitive/superior || **Interpretability** | High (physical basis) | Low (black box) || **Extrapolation** | Reasonable | Limited || **Novel climates** | Possible | Uncertain || **Development** | Decades of refinement | Rapid iteration |#### Performance Metrics**Weather Forecasting (GraphCast paper results):**- **Skill score vs ECMWF IFS**: GraphCast wins on 90% of targets at 10-day lead- **Tropical cyclones**: Better track forecasting than operational models- **Atmospheric rivers**: Improved prediction of extreme precipitation- **Upper atmosphere**: Superior stratospheric forecasts**Key Results:**- 500 hPa geopotential (weather patterns): ~10% better RMSE at day 5- Surface temperature: Competitive with best physics models- Precipitation: Good skill, some systematic biases- Extremes: Better than GCMs for many metrics#### Application to Climate Change**Direct Application:**- GraphCast is trained on current climate- Cannot directly simulate future climates (e.g., +4°C)**Potential Uses:**1. **Downscaling**: Take coarse GCM output → produce fine-scale patterns2. **Bias Correction**: Correct systematic GCM errors3. **Emulation**: Fast surrogate for expensive GCM runs4. **Process Studies**: Identify patterns in climate data5. **Hybrid Models**: ML components within physics-based frameworks**Climate Model Emulation:**- Train ML model on GCM output (thousands of years)- Emulator runs 1000× faster than GCM- Enables massive ensembles, sensitivity studies- Uncertainty quantification**Future Directions:**- **Climate GraphCast**: Train on multi-decade simulations spanning climate change- **Physics-informed ML**: Enforce conservation laws as constraints- **Uncertainty quantification**: Ensemble methods, Bayesian approaches- **Extreme events**: Specialized training for rare but important events#### Implementation Considerations**Computational Requirements:**- Training: Weeks on TPU v4 pods (expensive!)- Inference: Single GPU sufficient, very fast- Memory: ~10 GB for model weights**Data Requirements:**- Petabytes of reanalysis data- Consistent, quality-controlled observations- Long time series for training**Reproducibility:**- Model weights publicly available- Code open-sourced (JAX implementation)- Can be fine-tuned for regional applications#### Philosophical ImplicationsGraphCast represents a fundamental question: **Do we need to understand physics to predict climate?****Traditional view**: Understanding → Equations → Simulation → Prediction**ML view**: Data → Patterns → Prediction (Understanding optional)**Reality**: Hybrid approach likely optimal- Use physics for constraints, conservation- Use ML for complex parameterizations (clouds, convection)- Combine strengths of both approaches**Climate Science Community Response:**- Excitement about potential- Caution about extrapolation- Active research on hybrid models- Debate on role of physical understanding

In [ ]:
# Model 5: GraphCast-Style ML Model (Conceptual Implementation)## Note: Full GraphCast requires massive datasets and computational resources.# Here we demonstrate the key concepts with a simplified version that learns# patterns in our simpler models.import torchimport torch.nn as nnclass SimpleGraphCastAnalog(nn.Module):    """    Simplified analog of GraphCast demonstrating key ML concepts        This is NOT the real GraphCast but shows the ML approach:    - Learn from data (our previous models)    - Map inputs → future states    - Neural network architecture    """        def __init__(self, n_features=5, hidden_dim=64):        super().__init__()                # Encoder: Map atmospheric state to latent space        self.encoder = nn.Sequential(            nn.Linear(n_features, hidden_dim),            nn.ReLU(),            nn.Linear(hidden_dim, hidden_dim),            nn.ReLU()        )                # Processor: Transform in latent space (like GNN message passing)        self.processor = nn.Sequential(            nn.Linear(hidden_dim, hidden_dim),            nn.ReLU(),            nn.Linear(hidden_dim, hidden_dim),            nn.ReLU(),            nn.Linear(hidden_dim, hidden_dim),            nn.ReLU()        )                # Decoder: Map back to atmospheric state        self.decoder = nn.Sequential(            nn.Linear(hidden_dim, hidden_dim),            nn.ReLU(),            nn.Linear(hidden_dim, n_features)        )        def forward(self, x):        """        Forward pass: current state → future state                Args:            x: [batch, n_features] current atmospheric state        Returns:            x_pred: [batch, n_features] predicted future state        """        # Encode        h = self.encoder(x)                # Process        h = self.processor(h)                # Decode          x_pred = self.decoder(h)                return x_pred# Demonstrate the concept with synthetic data from our modelsprint("="*70)print("GRAPHCAST-STYLE ML MODEL (Conceptual Demonstration)")print("="*70 + "\n")print("Key Concepts Demonstrated:")print("  1. Data-driven approach: Learn from historical simulations")print("  2. Neural network architecture: Encoder-Processor-Decoder")print("  3. Pattern recognition: No explicit physics equations")print("  4. Fast inference: Once trained, predictions are rapid\n")# Create simple synthetic training data# Features: [T_surface, T_upper, gradient, wind, humidity]np.random.seed(42)n_samples = 1000# Current statesX_current = np.random.randn(n_samples, 5)X_current[:, 0] = 288 + 10*np.random.randn(n_samples)  # Surface T around 288KX_current[:, 1] = 250 + 10*np.random.randn(n_samples)  # Upper T around 250KX_current[:, 2] = X_current[:, 0] - X_current[:, 1]    # GradientX_current[:, 3] = 10 * np.random.randn(n_samples)       # WindX_current[:, 4] = 0.5 + 0.1*np.random.randn(n_samples) # Humidity# Future states (simplified evolution - in reality would come from GCM)X_future = X_current.copy()X_future[:, 0] += 0.01 * X_current[:, 2] + 0.1*np.random.randn(n_samples)  # T evolves with gradientX_future[:, 1] += 0.005 * X_current[:, 2] + 0.1*np.random.randn(n_samples)X_future[:, 2] = X_future[:, 0] - X_future[:, 1]X_future[:, 3] += 0.1 * X_current[:, 2] + np.random.randn(n_samples)  # Wind responds to gradientX_future[:, 4] += 0.001 * X_current[:, 0] - 0.5 + 0.01*np.random.randn(n_samples)  # Humidity# Convert to PyTorch tensorsX_train = torch.FloatTensor(X_current[:800])y_train = torch.FloatTensor(X_future[:800])X_test = torch.FloatTensor(X_current[800:])y_test = torch.FloatTensor(X_future[800:])print("Training Data:")print(f"  Training samples: {len(X_train)}")print(f"  Test samples: {len(X_test)}")print(f"  Features: 5 (T_surface, T_upper, gradient, wind, humidity)\n")# Initialize modelmodel = SimpleGraphCastAnalog(n_features=5, hidden_dim=64)# Count parametersn_params = sum(p.numel() for p in model.parameters())print(f"Model Architecture:")print(f"  Parameters: {n_params:,}")print(f"  Layers: Encoder (2) + Processor (3) + Decoder (2)")print(f"  Hidden dimension: 64\n")# Trainingprint("Training model (simplified)...")optimizer = torch.optim.Adam(model.parameters(), lr=0.001)criterion = nn.MSELoss()losses = []for epoch in range(100):    model.train()    optimizer.zero_grad()        # Forward    y_pred = model(X_train)    loss = criterion(y_pred, y_train)        # Backward    loss.backward()    optimizer.step()        losses.append(loss.item())        if (epoch + 1) % 20 == 0:        print(f"  Epoch {epoch+1}/100, Loss: {loss.item():.6f}")# Evaluationmodel.eval()with torch.no_grad():    y_pred_test = model(X_test)    test_loss = criterion(y_pred_test, y_test)print(f"\nTest Performance:")print(f"  Test Loss (MSE): {test_loss.item():.6f}")# Calculate skill metricswith torch.no_grad():    # Prediction skill    y_pred_np = y_pred_test.numpy()    y_test_np = y_test.numpy()        # Persistence forecast (baseline: no change)    X_test_np = X_test.numpy()    persistence_error = np.mean((X_test_np - y_test_np)**2)    ml_error = np.mean((y_pred_np - y_test_np)**2)        skill_score = 1 - ml_error / persistence_error        print(f"  Persistence Error: {persistence_error:.6f}")    print(f"  ML Model Error: {ml_error:.6f}")    print(f"  Skill Score: {skill_score:.4f}")    print(f"  ({skill_score*100:.1f}% improvement over persistence)\n")print("="*70)print("REAL GRAPHCAST CHARACTERISTICS")print("="*70)print("\n1. Scale: 37 million parameters (vs our {n_params:,})")print("  Real model is 1000× more complex")print("\n2. Training Data: 40+ years of global reanalysis")print("  Petabytes of atmospheric data")print("\n3. Performance: 10-day forecasts in ~1 minute")print("  Traditional GCMs take hours on supercomputers")print("\n4. Accuracy: Beats ECMWF operational model on 90% of metrics")print("  Particularly strong for extreme events")print("\n5. Applications:")print("  - Weather forecasting (operational use starting)")print("  - Climate model emulation (active research)")print("  - Downscaling coarse GCM output")print("  - Bias correction of climate projections")print("\n" + "="*70)

<a id='climate-change'></a>## Climate Change Analysis: Using Models to Understand Warming### Synthesis Across ModelsWe've built five models of increasing sophistication. Now we use them together to understand climate change, demonstrating how each contributes to our understanding.#### Key Questions We Can Answer:1. **How much will Earth warm with doubled CO₂?** (Climate Sensitivity)2. **Where will warming be strongest?** (Spatial Patterns)3. **How fast will warming occur?** (Transient Response)4. **What are the key feedbacks?** (Physical Mechanisms)5. **How certain are we?** (Model Agreement and Uncertainty)### Model Predictions Summary| Model | ECS (°C) | Key Features | Limitations ||-------|----------|--------------|-------------|| **1: 0D EBM** | ~1.2 | Global mean only | No feedbacks || **2: 1D RCM** | ~2.0 | Vertical structure | No geography || **3: 2D EBM** | ~2.8 | Polar amplification | No dynamics || **4: 3D GCM** | ~3.2 | Full spatial detail | Parameterizations || **5: GraphCast** | Data-driven | ML patterns | Extrapolation limited |**IPCC AR6 Assessment: ECS = 2.5-4.0°C (likely range), best estimate 3.0°C**Our progression shows convergence toward the observational estimate as we add complexity!### Physical Insights**Why Models Agree:**1. **Energy Balance**: All conserve energy2. **Greenhouse Effect**: CO₂ absorbs infrared radiation3. **Planck Response**: Warmer Earth emits more radiation4. **Water Vapor Feedback**: Warmer air holds more H₂O (greenhouse gas)**Why Models Differ:**1. **Ice-Albedo Feedback**: Requires geography (Models 3-4)2. **Cloud Feedback**: Complex, different parameterizations (GCMs)  3. **Lapse Rate Feedback**: Requires vertical structure (Models 2-4)4. **Regional Patterns**: Affect global mean through nonlinearities### Justifying Climate Change Projections#### Evidence from Models:**1. Model-Observation Agreement (Historical Period)**- All models successfully reproduce 20th century warming (~1°C)- Spatial patterns match (land>ocean, Arctic>tropics)- Cannot explain warming without human emissions**2. Physical Understanding**- Greenhouse effect is basic physics (known since 1896)- CO₂ absorbs at 15 μm (well-measured)- Increased CO₂ → reduced OLR → warming (unavoidable)**3. Multiple Lines of Evidence**- Paleoclimate: Past CO₂-temperature relationship- Satellite observations: Radiative forcing measured directly- Process studies: Individual feedbacks constrained- Model hierarchy: Simple to complex models agree**4. Consistency Across Scales**- Global mean temperature: All models converge- Regional patterns: Polar amplification robust- Seasonal cycle: Maintained in future- Extreme events: Intensification predicted#### Uncertainty Quantification**Sources of Uncertainty:**1. **Future Emissions** (Scenario Uncertainty):   - Depends on policy, technology, economics   - Range: +1.5°C to +4.5°C by 2100   - Largest source of uncertainty2. **Climate Response** (Model Uncertainty):   - Cloud feedbacks: ±0.5°C   - Carbon cycle: ±0.3°C   - Ice sheets: ±0.2°C   - Total: ±0.7°C3. **Natural Variability** (Internal Variability):   - ENSO, volcanoes, solar: ±0.2°C on decadal scales   - Averages out over longer periods**Confidence Levels (IPCC AR6):**- Human influence on warming: **Unequivocal** (100%)- Continued warming with emissions: **Virtually certain** (>99%)- Exceeding 1.5°C by 2040: **Very likely** (>90%)- Warming continues for centuries: **Very high confidence** (>95%)### Policy-Relevant Findings**What We Know with High Confidence:**✓ Each ton of CO₂ causes warming (linearly)✓ Warming committed even if emissions stop✓ Limiting warming requires net-zero emissions✓ Earlier action is cheaper and more effective✓ Impacts scale with warming magnitude**What Remains Uncertain:**? Exact magnitude of warming (2.5-4°C range for 2×CO₂)? Regional precipitation changes (sign and magnitude)? Tipping points and abrupt changes (ice sheets, AMOC)? Climate-carbon cycle feedbacks (permafrost, forests)? Exact timing of impacts**Key Message:**Uncertainty is NOT a reason for inaction - it includes possibilities of outcomes worse than best estimates!

In [ ]:
# Comprehensive Climate Change Analysis Using All Modelsprint("="*80)print("CLIMATE CHANGE ANALYSIS: SYNTHESIS ACROSS ALL MODELS")print("="*80 + "\n")# Compile climate sensitivity resultsmodels = ['Model 1\n0D EBM', 'Model 2\n1D RCM', 'Model 3\n2D EBM',           'Model 4\n3D GCM', 'IPCC AR6\nBest Est.']ECS_values = [    model1.climate_sensitivity(),    ECS,    ECS_2d,    global_mean_warming * (3.7/4.0),  # Scale to standard forcing    3.0  # IPCC best estimate]print("Climate Sensitivity (°C per doubling of CO₂):")print("-" * 60)for model_name, ecs in zip(models, ECS_values):    bar = '█' * int(ecs * 10)    print(f"{model_name:20s}: {ecs:4.1f}°C {bar}")print("-" * 60 + "\n")# Create comprehensive comparison figurefig = plt.figure(figsize=(18, 10))gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)# === Panel 1: Climate Sensitivity Comparison ===ax1 = fig.add_subplot(gs[0, 0])x_pos = np.arange(len(models))colors_bar = ['lightblue', 'skyblue', 'cornflowerblue', 'royalblue', 'darkred']bars = ax1.bar(x_pos, ECS_values, color=colors_bar, edgecolor='black', linewidth=2, alpha=0.8)# Add IPCC rangeax1.axhspan(2.5, 4.0, alpha=0.2, color='gray', label='IPCC AR6 likely range')ax1.axhline(3.0, color='red', linestyle='--', linewidth=2, alpha=0.7, label='IPCC best estimate')# Add value labelsfor bar, val in zip(bars, ECS_values):    height = bar.get_height()    ax1.text(bar.get_x() + bar.get_width()/2., height,             f'{val:.1f}°C', ha='center', va='bottom', fontsize=11, fontweight='bold')ax1.set_ylabel('Climate Sensitivity (°C)', fontsize=13, fontweight='bold')ax1.set_title('Model Convergence on Climate Sensitivity', fontsize=14, fontweight='bold')ax1.set_xticks(x_pos)ax1.set_xticklabels([m.replace('\n', ' ') for m in models], rotation=45, ha='right')ax1.legend(fontsize=10)ax1.grid(True, axis='y', alpha=0.3)ax1.set_ylim(0, 5)# === Panel 2: Warming Patterns (Spatial) ===ax2 = fig.add_subplot(gs[0, 1])# Use 2D model warming patternwarming_pattern_2d = T_forced_2d - T_control_2dax2.plot(model3.lat, warming_pattern_2d, 'red', linewidth=3, marker='o', markersize=6)ax2.axhline(ECS_2d, color='gray', linestyle='--', linewidth=2, alpha=0.7,             label=f'Global mean: {ECS_2d:.2f}°C')ax2.fill_between(model3.lat, ECS_2d, warming_pattern_2d,                   where=(warming_pattern_2d > ECS_2d),                  alpha=0.3, color='red', label='Enhanced warming')ax2.fill_between(model3.lat, ECS_2d, warming_pattern_2d,                  where=(warming_pattern_2d < ECS_2d),                  alpha=0.3, color='blue', label='Reduced warming')ax2.set_xlabel('Latitude (°)', fontsize=12, fontweight='bold')ax2.set_ylabel('Temperature Change (K)', fontsize=12, fontweight='bold')ax2.set_title('Polar Amplification Pattern', fontsize=14, fontweight='bold')ax2.legend(fontsize=10)ax2.grid(True, alpha=0.3)ax2.set_xlim(-90, 90)# === Panel 3: Feedback Contributions ===ax3 = fig.add_subplot(gs[0, 2])feedbacks = ['Planck\nResponse', 'Water\nVapor', 'Lapse\nRate', 'Ice-\nAlbedo', 'Clouds', 'Net']feedback_values = [-3.2, +1.8, -0.5, +0.4, +0.6, -0.9]  # W/m²/K (typical values)feedback_colors = ['blue', 'red', 'lightblue', 'cyan', 'gray', 'darkgreen']bars_fb = ax3.barh(feedbacks, feedback_values, color=feedback_colors,                     edgecolor='black', linewidth=1.5, alpha=0.7)ax3.axvline(0, color='black', linewidth=2)ax3.set_xlabel('Feedback Parameter (W/m²/K)', fontsize=12, fontweight='bold')ax3.set_title('Climate Feedback Analysis', fontsize=14, fontweight='bold')ax3.grid(True, axis='x', alpha=0.3)# Add annotationsfor bar, val in zip(bars_fb, feedback_values):    width = bar.get_width()    ax3.text(width, bar.get_y() + bar.get_height()/2.,             f' {val:+.1f}', ha='left' if val > 0 else 'right',             va='center', fontsize=10, fontweight='bold')# === Panel 4: Transient vs Equilibrium Response ===ax4 = fig.add_subplot(gs[1, 0])# Simulate transient responseyears_future = np.arange(0, 150)forcing_trajectory = np.minimum(3.7 * years_future / 70, 3.7)  # Reach 2xCO2 in 70 years# Use Model 1 for transient responseT_transient = []T_current = 288for year, F in zip(years_future, forcing_trajectory):    if year == 0:        T_transient.append(T_current)    else:        dT = model1.energy_balance(T_current, year, F) * 0.5  # Slower response        T_current += dT        T_transient.append(T_current)T_transient = np.array(T_transient)T_equilibrium = np.array([model1.equilibrium_temperature(F) for F in forcing_trajectory])ax4.plot(years_future, T_transient - 288, 'blue', linewidth=3, label='Transient Response')ax4.plot(years_future, T_equilibrium - 288, 'red', linewidth=3, linestyle='--',          label='Equilibrium Response')ax4.fill_between(years_future, T_transient - 288, T_equilibrium - 288,                  alpha=0.3, color='orange', label='Committed Warming')ax4.set_xlabel('Year', fontsize=12, fontweight='bold')ax4.set_ylabel('Temperature Anomaly (°C)', fontsize=12, fontweight='bold')ax4.set_title('Transient vs Equilibrium Warming', fontsize=14, fontweight='bold')ax4.legend(fontsize=10, loc='upper left')ax4.grid(True, alpha=0.3)ax4.set_xlim(0, 150)# === Panel 5: Cumulative Emissions vs Warming ===ax5 = fig.add_subplot(gs[1, 1])# Approximate relationship: ~0.5°C per 1000 GtCO2cumulative_emissions = np.linspace(0, 5000, 100)  # GtCO2warming_from_emissions = cumulative_emissions * 0.00045  # °C per GtCO2ax5.plot(cumulative_emissions, warming_from_emissions, 'darkred', linewidth=4)# Add markers for key thresholdsax5.axhline(1.5, color='orange', linestyle='--', linewidth=2, label='1.5°C target')ax5.axhline(2.0, color='red', linestyle='--', linewidth=2, label='2.0°C target')# Mark current emissionscurrent_cumulative = 2400  # Approximate historicalcurrent_warming = current_cumulative * 0.00045ax5.plot(current_cumulative, current_warming, 'ko', markersize=15, label='Current (~2023)')# Annotateax5.text(current_cumulative + 200, current_warming,          f'Current\n{current_cumulative} GtCO₂\n{current_warming:.1f}°C',         fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))ax5.set_xlabel('Cumulative CO₂ Emissions (GtCO₂)', fontsize=12, fontweight='bold')ax5.set_ylabel('Global Warming (°C)', fontsize=12, fontweight='bold')ax5.set_title('Carbon Budget: Emissions → Warming', fontsize=14, fontweight='bold')ax5.legend(fontsize=10, loc='upper left')ax5.grid(True, alpha=0.3)ax5.set_xlim(0, 5000)ax5.set_ylim(0, 2.5)# === Panel 6: Uncertainty Ranges ===ax6 = fig.add_subplot(gs[1, 2])scenarios = ['Low\nEmissions', 'Medium\nEmissions', 'High\nEmissions']warming_2100 = [1.8, 2.7, 4.4]  # Best estimatesuncertainty_low = [1.3, 2.1, 3.3]uncertainty_high = [2.4, 3.5, 5.7]x_pos_sc = np.arange(len(scenarios))ax6.bar(x_pos_sc, warming_2100, color=['green', 'orange', 'red'],        edgecolor='black', linewidth=2, alpha=0.7)# Add uncertainty rangesfor i, (low, mid, high) in enumerate(zip(uncertainty_low, warming_2100, uncertainty_high)):    ax6.plot([i, i], [low, high], 'k-', linewidth=3)    ax6.plot([i-0.1, i+0.1], [low, low], 'k-', linewidth=2)    ax6.plot([i-0.1, i+0.1], [high, high], 'k-', linewidth=2)        # Add labels    ax6.text(i, high + 0.2, f'{high:.1f}°C', ha='center', fontsize=10, fontweight='bold')    ax6.text(i, mid, f'{mid:.1f}°C', ha='center', va='center',              fontsize=11, fontweight='bold', color='white',             bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))    ax6.text(i, low - 0.2, f'{low:.1f}°C', ha='center', fontsize=10, fontweight='bold')# Add targetsax6.axhline(1.5, color='orange', linestyle='--', linewidth=2, alpha=0.5)ax6.axhline(2.0, color='red', linestyle='--', linewidth=2, alpha=0.5)ax6.set_ylabel('Warming by 2100 (°C)', fontsize=12, fontweight='bold')ax6.set_title('Projected Warming Under Different Scenarios', fontsize=14, fontweight='bold')ax6.set_xticks(x_pos_sc)ax6.set_xticklabels([s.replace('\n', ' ') for s in scenarios])ax6.grid(True, axis='y', alpha=0.3)ax6.set_ylim(0, 6)plt.suptitle('Climate Change: Model Synthesis and Projections',             fontsize=18, fontweight='bold', y=0.995)plt.savefig('climate_change_synthesis.png', dpi=150, bbox_inches='tight')plt.show()print("\n" + "="*80)print("KEY FINDINGS: CLIMATE CHANGE ANALYSIS")print("="*80)print("\n1. CLIMATE SENSITIVITY")print(f"   • Simple models (0D-1D): {model1.climate_sensitivity():.1f}-{ECS:.1f}°C - underestimate")print(f"   • Complex models (2D-3D): {ECS_2d:.1f}-{global_mean_warming * (3.7/4.0):.1f}°C - match observations")print(f"   • IPCC Assessment: 2.5-4.0°C (likely), 3.0°C (best estimate)")print("   • Model hierarchy shows convergence with added physics")print("\n2. SPATIAL PATTERNS")print("   • Polar amplification: Arctic warms 2-3× faster than global mean")print("   • Land warms faster than ocean (lower heat capacity)")print("   • Tropics show moderate warming but severe humidity impacts")print("   • Regional patterns critical for impacts assessment")print("\n3. FEEDBACKS")print("   • Water vapor: Strongly positive (+1.8 W/m²/K)")print("   • Ice-albedo: Positive at high latitudes (+0.4 W/m²/K)")print("   • Clouds: Uncertain but likely positive (+0.6 W/m²/K)")print("   • Lapse rate: Negative feedback (-0.5 W/m²/K)")print("   • Net feedback parameter: -0.9 W/m²/K → ECS ≈ 3°C")print("\n4. TRANSIENT RESPONSE")print("   • Warming lags forcing due to ocean thermal inertia")print("   • ~40% of equilibrium warming realized after 70 years")print("   • Committed warming even if emissions stopped today")print("   • Full equilibrium takes centuries to millennia")print("\n5. CARBON BUDGET")print("   • Nearly linear relationship: ~0.45°C per 1000 GtCO₂")print(f"   • Current emissions: ~{current_cumulative} GtCO₂ → {current_warming:.1f}°C warming")print("   • 1.5°C budget: ~500 GtCO₂ remaining (at current emissions: ~12 years)")print("   • 2.0°C budget: ~1200 GtCO₂ remaining (~30 years)")print("\n6. FUTURE SCENARIOS")print("   • Low emissions (SSP1-2.6): 1.3-2.4°C by 2100")print("   • Medium emissions (SSP2-4.5): 2.1-3.5°C by 2100")print("   • High emissions (SSP5-8.5): 3.3-5.7°C by 2100")print("   • Every fraction of degree matters for impacts")print("\n7. CONFIDENCE ASSESSMENT")print("   • Human-caused warming: Unequivocal (100% certain)")print("   • Continued warming with emissions: Virtually certain (>99%)")print("   • Magnitude of future warming: Likely range well-constrained")print("   • Regional details: Moderate confidence")print("   • Extreme events: Growing evidence base")print("\n" + "="*80)

## Conclusions and Summary### Journey Through Climate ModelsWe've progressed through five generations of climate modeling, each adding layers of sophistication:1. **Model 1 (0D EBM)**: Established energy balance fundamentals2. **Model 2 (1D RCM)**: Added vertical atmospheric structure3. **Model 3 (2D EBM)**: Incorporated meridional variations and ice-albedo feedback4. **Model 4 (3D GCM)**: Full three-dimensional dynamics and circulation5. **Model 5 (GraphCast)**: Machine learning-based pattern recognition### Key Takeaways**Scientific Understanding:**- Climate change is rooted in basic physics (energy balance, greenhouse effect)- Multiple independent lines of evidence converge on similar conclusions- Model hierarchy builds confidence through consistency- Uncertainty does not imply lack of knowledge - ranges are well-constrained**Technical Insights:**- Simple models provide intuition and rapid exploration- Complex models capture essential regional details- Machine learning offers new approaches but doesn't replace physics- All models have limitations - use appropriate tool for question**Policy Implications:**- Warming is proportional to cumulative emissions- Net-zero emissions required to stabilize temperature- Earlier action is more effective and less costly- Every tenth of a degree matters for impacts### Future Directions**Model Development:**- Higher resolution (km-scale globally)- Better representation of clouds and precipitation- Improved ice sheet dynamics- Interactive carbon cycle and vegetation- Hybrid physics-ML approaches**Scientific Challenges:**- Tipping points and abrupt changes- Regional climate change and extremes- Multi-century sea level rise- Climate-carbon cycle feedbacks- Attribution of specific events**Applications:**- Climate services for adaptation planning- Early warning systems for extremes- Impact assessments (agriculture, water, health)- Policy evaluation and carbon budgets- Long-term planning (infrastructure, insurance)### Final ThoughtsClimate models, from the simplest energy balance to the most sophisticated machine learning systems, all tell the same fundamental story: **Earth's climate is sensitive to greenhouse gas concentrations, and continued emissions will cause substantial warming with serious consequences.**The progression from Model 1 to Model 5 demonstrates that this conclusion is robust across modeling approaches, physical understanding, and mathematical frameworks. While uncertainties remain in details, the big picture is clear and demands action.**As physicist Richard Feynman said: "Nature uses only the longest threads to weave her patterns, so that each small piece of her fabric reveals the organization of the entire tapestry."**Our hierarchy of models reveals this tapestry, from the simplest threads of energy balance to the complex weave of global circulation and the learned patterns of machine intelligence.---### References and Further Reading**Key Papers:**- Budyko (1969): Simple climate model foundations- Manabe & Wetherald (1975): First 3D climate model with CO₂ doubling- Cess et al. (1989): Climate feedback analysis- IPCC AR6 WG1 (2021): Comprehensive assessment- Lam et al. (2023): GraphCast paper (Nature)**Textbooks:**- Hartmann: "Global Physical Climatology"- Marshall & Plumb: "Atmosphere, Ocean, and Climate Dynamics"- Peixoto & Oort: "Physics of Climate"- McGuffie & Henderson-Sellers: "A Climate Modelling Primer"**Online Resources:**- CMIP6 model archive: https://esgf-node.llnl.gov/- ERA5 reanalysis: https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5- GraphCast code: https://github.com/deepmind/graphcast- IPCC Reports: https://www.ipcc.ch/---*Thank you for following this journey through climate modeling!*